In [ ]:
import os, sys
sys.path.insert(1, './codes/codes')  # insert at 1, 0 is the script path (or '' in REPL)

print("----------------")
!python --version
!nvidia-smi
print("----------------")
print("System Version: ", sys.version)

## ======================================================== ##
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from plot_functions import add_colorbar, imagesc

print("PyTorch Version: ", torch.__version__)
print("----------------")

print("torch.cuda.is_available: ",torch.cuda.is_available())
print("----------------")
print(torch.__version__, torch.version.cuda, torch.cuda.get_device_name(0))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# deterministic behavior
torch.manual_seed(3)
torch.cuda.manual_seed_all(3)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(3)
# random.seed(3)
os.environ['PYTHONHASHSEED'] = str(3)

In [ ]:
import pandas as pd
from scipy.ndimage import gaussian_filter

# Load the velocity model 
vmodel = np.array(pd.read_csv("./vel_marmousi_376x1151.csv")) 
# v_init = np.array(pd.read_csv("./vel_marmousi_smooth400_376x1151.csv"))
v_init = np.array(pd.read_csv("./vel_marmousi_376x1151.csv"))
# v_init = gaussian_filter(v_init, sigma=10)
v_init = gaussian_filter(v_init, sigma=15)

dz = 8
nz, nx = vmodel.shape
print("Original Model Shape: {}, Grid Interval: {}m".format(vmodel.shape, dz))

################# Plot true & initial velocity model #################
fig = plt.figure(figsize=(12, 2))
gs = fig.add_gridspec(1, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]")
im = ax.imshow(vmodel/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

ax = fig.add_subplot(gs[0, 1])
ax.set( xlabel="Distance x[km]")
im = ax.imshow(v_init/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
################# Plot true & initial velocity model #################


dz = 15
vp_tensor_init = torch.from_numpy(vmodel).type(dtype=torch.float32).to(device)
vi_tensor_init = torch.from_numpy(v_init).type(dtype=torch.float32).to(device)
sea = (torch.ones([9,1151])*5500).type(dtype=torch.float32).to(device)
vp_tensor = torch.cat([vp_tensor_init, sea], axis=0)[None, ::3, 60:1084:4]
vi_tensor = torch.cat([vp_tensor_init, sea], axis=0)[None, ::3, 60:1084:4]
vi_tensor = torch.from_numpy(gaussian_filter(vi_tensor.cpu().numpy(), sigma=10)).type(dtype=torch.float32).to(device)

nv, nz, nx = vp_tensor.shape
print("Resampled Model Shape: {}, Grid Interval: {}m".format((nz, nx), dz))

# Setting locations of sources and receivers
xs = torch.arange(20, nx-10, 20, dtype=torch.long).repeat([nv, 1])      # x-coordinate for sources
ns = xs.shape[1]                                                        # number of shots 
xr = torch.arange(0, nx, 1, dtype=torch.long).repeat([nv, ns, 1])       # x-coordinate for receivers
zs = torch.full((nv, ns), 0, dtype=torch.long)                          # depth of sources    震源深度
zr = torch.full((nv, ns, nx), 0, dtype=torch.long)                      # depth of receivers 地表
print("Number of shots: {}, with interval: {}m, in depth: {}m".format(ns, (xs[0, 1]-xs[0, 0])*dz, zs[0, 0]*dz))
print(xs)

################# Plot true & initial velocity model #################
fig = plt.figure(figsize=(12, 2))
gs = fig.add_gridspec(1, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]")
im = ax.imshow(vp_tensor[0].cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

ax = fig.add_subplot(gs[0, 1])
ax.set( xlabel="Distance x[km]")
im = ax.imshow(vi_tensor[0].cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# ################# Plot true & initial velocity model #################

In [ ]:
torch.save(vi_tensor, './Figures/v_samp/vi_tensor_noscale.pth')
torch.save(vp_tensor, './Figures/v_samp/vp_tensor_noscale.pth')

In [ ]:
xr = torch.save(xr, './Figures/xr.npz')
xs = torch.save(xs, './Figures/xs.npz')

In [ ]:
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D

# Load the velocity model and its initial 
file = np.load("./SEGoverthrust.npz")
vel3d = file['vmodel'].transpose(1, 0, 2)/1000  #z, x, y
x = file['x']
y = file['y']
z = file['z']
dx = file['dx']/1000
dy = file['dy']/1000
dz = file['dz']/1000
nx = 256*3
ny = 256*3
nz = 62*3
dx = 24/1000/3
dy = 24/1000/3
dz = 24/1000/3
# nz, nx, ny = vel3d.shape

# print("Geological Model Shape: {}".format(vel3d.shape))
# print("Min & Max: Vp[{:.4f}, {:.4f}]]".format(vel3d.min(), vel3d.max()))
# print("Mean & Std: Vp[{:.4f}, {:.4f}]]".format(vel3d.mean(), vel3d.std()))

fig = plt.figure(figsize=(18, 3))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel("Depth [km]", fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(vel3d[0:186, 100, 0:768], vmin=2, vmax=6, extent=[0, nx*dx, nz*dz, 0], aspect=2, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12)
# ax.set_yticks([])
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# cbar.ax.tick_params(labelsize=12)  
# cbar.set_label('km/s', size=14) 


ax = fig.add_subplot(gs[0, 1])
# ax.set_ylabel("Depth [km]", fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(vel3d[0:186, 0:768, 400], vmin=2, vmax=6, extent=[0, ny*dy, nz*dz, 0], aspect=2, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12)
ax.set_yticks([])
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 


ax2 = fig.add_subplot(gs[0, 2])
ax2.set_ylabel("Distance [km]", fontsize=16)
ax2.set_xlabel('Distance [km]', fontsize=16)
ax2.set_title('(c)', loc='center', fontsize=20)
im2 = ax2.imshow(vel3d[70, 0:768, 0:768], vmin=2, vmax=6, extent=[0, nx*dx, ny*dy, 0], aspect=1, cmap='RdBu_r')
ax2.tick_params(axis='x', labelsize=12) 
ax2.tick_params(axis='y', labelsize=12)
# ax.set_yticks([])
cbar = add_colorbar(ax2, im2, ax2.transAxes, width="5%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

# cbar = add_colorbar(ax, im, ax.transAxes, width="10%", bbox_to_anchor=(1.05, 0.0, 1, 1), ctitle='$km/s$')

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.06,
                    hspace=0)

plt.savefig('./Figures/figures/overthrust.png', dpi=300, bbox_inches='tight')

In [ ]:
deeper_yellow = (1.0, 0.85, 0.0) 

################# Plot true & initial velocity model #################
fig = plt.figure(figsize=(12, 3))
gs = fig.add_gridspec(1, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel("Depth [km]", fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
im = ax.imshow(vp_tensor[0].cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
plt.scatter(0.6, 0.45, s=200, alpha=1, marker="*", facecolor='orange', label='True')
plt.scatter(1.8, 1.35, s=200, alpha=1, marker="*", facecolor='red', label='True')
plt.scatter(2.4, 1.05, s=200, alpha=1, marker="*", facecolor='green', label='True')
plt.scatter(3.75, 1.8, s=200, alpha=1, marker="*", facecolor='blue', label='True')

ax = fig.add_subplot(gs[0, 1])
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(vi_tensor[0].cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=12) 
ax.set_yticks([])
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,
                    hspace=0.4)

plt.savefig('./Figures/figures/marmousi.png', dpi=300, bbox_inches='tight')

In [ ]:
v_samp_AE = torch.load("./Figures/v_samp/v_pred_AE_3000.pth").cpu()/1000
v_samp_VAE = torch.load("./Figures/v_samp/v_samp_eval_VAE_N1.pth").cpu()/1000
v_samp_Dropout = torch.load("./Figures/v_samp/v_samp_train_Dropout_p0.1.pth").cpu()/1000
v_samp_eval_Dropout = torch.load("./Figures/v_samp/v_pred_Dropout_3000.pth").cpu()/1000



In [ ]:
30,40
90,120
70,160
120,250

In [ ]:
from scipy.stats import norm


fig = plt.figure(figsize=(30, 3))
gs = fig.add_gridspec(1, 5)
ax = fig.add_subplot(gs[0, 0])
# 定义均值和方差
zz = 30
xx = 40
vmean_VAE = torch.mean(v_samp_VAE[:, zz, xx])
vstd_VAE = torch.std(v_samp_VAE[:, zz, xx])
vmean_Drop = torch.mean(v_samp_Dropout[:, zz, xx])
vstd_Drop = torch.std(v_samp_Dropout[:, zz, xx])
x_AE_scat = v_samp_AE[:, zz, xx].detach().numpy() 
x_Drop_scat = v_samp_eval_Dropout[:, zz, xx].detach().numpy() 
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
# 创建正态分布
normal_distribution_VAE = torch.distributions.Normal(vmean_VAE, vstd_VAE)
normal_distribution_Drop = torch.distributions.Normal(vmean_Drop, vstd_Drop)
# normal_distribution_init = torch.distributions.Normal(2900, 1)
# 生成一些样本
samples_VAE = normal_distribution_VAE.sample((3000,))
samples_Drop = normal_distribution_Drop.sample((3000,))
# samples_init = normal_distribution_init.sample((3000,))
# 生成 x 值
x_VAE = np.linspace(-1000, 5000, 3000)/1000
x_Drop = np.linspace(-1000, 5000, 3000)/1000
# 计算正态分布的概率密度函数值
pdf_VAE = norm.pdf(x_VAE, vmean_VAE.detach().numpy(), vstd_VAE.detach().numpy())
pdf_Drop = norm.pdf(x_Drop, vmean_Drop.detach().numpy(), vstd_Drop.detach().numpy())
# pdf_init = norm.pdf(x, 2900, 1)
# 画出分布
plt.hist(samples_VAE.numpy(), bins=50, density=True, alpha=0.6, color='green') #bins代表箱子的个数
plt.hist(samples_Drop.numpy(), bins=50, density=True, alpha=0.6, color='orange') 
# plt.hist(samples_init.numpy(), bins=50, density=True) 
plt.plot(x_Drop, pdf_Drop, label='Dropout', color='darkorange')
plt.plot(x_VAE, pdf_VAE, label='VAE', color='darkgreen')
# plt.plot(x, pdf_init, label='init Distribution')
y_scat = np.ones(1)*0.6
plt.scatter(x_AE_scat, y_scat, s=108, alpha=1, marker="o", color='blue', label='AE')
# plt.scatter(x_Drop_scat, y_scat, s=108, alpha=1, marker="s", color='m', label='Dropout off')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.set_xlabel("Velocity [km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
plt.title('(a)', loc='center', fontsize=20)


ax = fig.add_subplot(gs[0, 1])
# 定义均值和方差
zz = 90
xx = 120
vmean_VAE = torch.mean(v_samp_VAE[:, zz, xx])
vstd_VAE = torch.std(v_samp_VAE[:, zz, xx])
vmean_Drop = torch.mean(v_samp_Dropout[:, zz, xx])
vstd_Drop = torch.std(v_samp_Dropout[:, zz, xx])
x_AE_scat = v_samp_AE[:, zz, xx].detach().numpy() 
x_Drop_scat = v_samp_eval_Dropout[:, zz, xx].detach().numpy() 
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
# 创建正态分布
normal_distribution_VAE = torch.distributions.Normal(vmean_VAE, vstd_VAE)
normal_distribution_Drop = torch.distributions.Normal(vmean_Drop, vstd_Drop)
# normal_distribution_init = torch.distributions.Normal(2900, 1)
# 生成一些样本
samples_VAE = normal_distribution_VAE.sample((3000,))
samples_Drop = normal_distribution_Drop.sample((3000,))
# samples_init = normal_distribution_init.sample((3000,))
# 生成 x 值
x_VAE = np.linspace(1300, 5600, 3000)/1000
x_Drop = np.linspace(1300, 5600, 3000)/1000
# 计算正态分布的概率密度函数值
pdf_VAE = norm.pdf(x_VAE, vmean_VAE.detach().numpy(), vstd_VAE.detach().numpy())
pdf_Drop = norm.pdf(x_Drop, vmean_Drop.detach().numpy(), vstd_Drop.detach().numpy())
# pdf_init = norm.pdf(x, 2900, 1)
# 画出分布
plt.hist(samples_VAE.numpy(), bins=50, density=True, alpha=0.6, color='green') #bins代表箱子的个数
plt.hist(samples_Drop.numpy(), bins=50, density=True, alpha=0.6, color='orange') 
# plt.hist(samples_init.numpy(), bins=50, density=True) 
plt.plot(x_VAE, pdf_VAE, label='VAE Distribution', color='darkgreen')
plt.plot(x_Drop, pdf_Drop, label='Dropout on Distribution', color='darkorange')
# plt.plot(x, pdf_init, label='init Distribution')
y_scat = np.ones(1)*0.15
plt.scatter(x_AE_scat, y_scat, s=108, alpha=1, marker="o", color='blue', label='AE prediction')
# plt.scatter(x_Drop_scat, y_scat, s=108, alpha=1, marker="s", color='m', label='Dropout off prediction')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.set_xlabel("Velocity [km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.legend(fontsize=12, ncol=1)
plt.title('(b)', loc='center', fontsize=20)

ax = fig.add_subplot(gs[0, 2])
# 定义均值和方差
zz = 70
xx = 160
vmean_VAE = torch.mean(v_samp_VAE[:, zz, xx])
vstd_VAE = torch.std(v_samp_VAE[:, zz, xx])
vmean_Drop = torch.mean(v_samp_Dropout[:, zz, xx])
vstd_Drop = torch.std(v_samp_Dropout[:, zz, xx])
x_AE_scat = v_samp_AE[:, zz, xx].detach().numpy() 
x_Drop_scat = v_samp_eval_Dropout[:, zz, xx].detach().numpy() 
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
# 创建正态分布
normal_distribution_VAE = torch.distributions.Normal(vmean_VAE, vstd_VAE)
normal_distribution_Drop = torch.distributions.Normal(vmean_Drop, vstd_Drop)
# normal_distribution_init = torch.distributions.Normal(2900, 1)
# 生成一些样本
samples_VAE = normal_distribution_VAE.sample((3000,))
samples_Drop = normal_distribution_Drop.sample((3000,))
# samples_init = normal_distribution_init.sample((3000,))
# 生成 x 值
x_VAE = np.linspace(1500, 3500, 3000)/1000
x_Drop = np.linspace(1500, 3500, 3000)/1000
# 计算正态分布的概率密度函数值
pdf_VAE = norm.pdf(x_VAE, vmean_VAE.detach().numpy(), vstd_VAE.detach().numpy())
pdf_Drop = norm.pdf(x_Drop, vmean_Drop.detach().numpy(), vstd_Drop.detach().numpy())
# pdf_init = norm.pdf(x, 2900, 1)
# 画出分布
plt.hist(samples_VAE.numpy(), bins=50, density=True, alpha=0.6, color='green') #bins代表箱子的个数
plt.hist(samples_Drop.numpy(), bins=50, density=True, alpha=0.6, color='orange') 
# plt.hist(samples_init.numpy(), bins=50, density=True) 
plt.plot(x_VAE, pdf_VAE, label='VAE Distribution', color='darkgreen')
plt.plot(x_Drop, pdf_Drop, label='Dropout on Distribution', color='darkorange')
# plt.plot(x, pdf_init, label='init Distribution')
y_scat = np.ones(1)*0.16
plt.scatter(x_AE_scat, y_scat, s=108, alpha=1, marker="o", color='blue', label='AE prediction')
# plt.scatter(x_Drop_scat, y_scat, s=108, alpha=1, marker="s", color='m', label='Dropout off prediction')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.set_xlabel("Velocity [km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.legend(fontsize=12, ncol=1)
plt.title('(c)', loc='center', fontsize=20)



ax = fig.add_subplot(gs[0, 3])
# 定义均值和方差
zz = 120
xx = 250
vmean_VAE = torch.mean(v_samp_VAE[:, zz, xx])
vstd_VAE = torch.std(v_samp_VAE[:, zz, xx])
vmean_Drop = torch.mean(v_samp_Dropout[:, zz, xx])
vstd_Drop = torch.std(v_samp_Dropout[:, zz, xx])
x_AE_scat = v_samp_AE[:, zz, xx].detach().numpy() 
x_Drop_scat = v_samp_eval_Dropout[:, zz, xx].detach().numpy() 
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
# 创建正态分布
normal_distribution_VAE = torch.distributions.Normal(vmean_VAE, vstd_VAE)
normal_distribution_Drop = torch.distributions.Normal(vmean_Drop, vstd_Drop)
# normal_distribution_init = torch.distributions.Normal(2900, 1)
# 生成一些样本
samples_VAE = normal_distribution_VAE.sample((3000,))
samples_Drop = normal_distribution_Drop.sample((3000,))
# samples_init = normal_distribution_init.sample((3000,))
# 生成 x 值
x_VAE = np.linspace(1500, 7000, 3000)/1000
x_Drop = np.linspace(1500, 7000, 3000)/1000
# 计算正态分布的概率密度函数值
pdf_VAE = norm.pdf(x_VAE, vmean_VAE.detach().numpy(), vstd_VAE.detach().numpy())
pdf_Drop = norm.pdf(x_Drop, vmean_Drop.detach().numpy(), vstd_Drop.detach().numpy())
# pdf_init = norm.pdf(x, 2900, 1)
# 画出分布
plt.hist(samples_VAE.numpy(), bins=50, density=True, alpha=0.6, color='green') #bins代表箱子的个数
plt.hist(samples_Drop.numpy(), bins=50, density=True, alpha=0.6, color='orange') 
# plt.hist(samples_init.numpy(), bins=50, density=True) 
plt.plot(x_VAE, pdf_VAE, label='VAE', color='darkgreen')
plt.plot(x_Drop, pdf_Drop, label='Dropout-on', color='darkorange')
# plt.plot(x, pdf_init, label='init Distribution')
y_scat = np.ones(1)*0.14
# plt.scatter(x_Drop_scat, y_scat, s=108, alpha=1, marker="s", color='m', label='Dropout-off')
plt.scatter(x_AE_scat, y_scat, s=108, alpha=1, marker="o", color='blue', label='AE')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.set_xlabel("Velocity [km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.legend(fontsize=12, ncol=1)
plt.title('(d)', loc='center', fontsize=20)

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.15,
                    hspace=0.4)

plt.savefig('./Figures/figures/all distribution.png', dpi=300, bbox_inches='tight')

# plt.show()

# VAE 示意图汇总

In [ ]:
# 汇总展示反演过程的vpred
epochs = [1, 200, 500, 1000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/VAE-G-inversion-results/v_pred_{}.pth'.format(epoch))
# print(v_pred)  

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 3))
gs = fig.add_gridspec(1, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.4)

plt.savefig('./Figures/figures/VAE-G-inversion-results.png', dpi=300, bbox_inches='tight')


In [ ]:
VAE_v_samp1 = torch.load('./Figures/v_samp/v_samp_eval_VAE_N1.pth').cpu()
VAE_v_samp100 = torch.load('./Figures/v_samp/v_samp_eval_VAE_N10.pth').cpu()
VAE_v_samp05 = torch.load('./Figures/v_samp/v_samp_eval_VAE_N05-3.pth').cpu()
VAE_v_samp01 = torch.load('./Figures/v_samp/v_samp_eval_VAE_N01.pth').cpu()

vi_tensor = torch.load('./Figures/v_samp/vi_tensor_noscale.pth').cpu()
vp_tensor = torch.load('./Figures/v_samp/vp_tensor_noscale.pth').cpu()

In [ ]:
# VAE 不确定性分析
cmap = 'RdBu_r'
fig = plt.figure(figsize=(24, 9))
gs = fig.add_gridspec(3, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp1, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp100, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp05, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp01, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 
#-------------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(torch.std(VAE_v_samp1, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp1, axis=0).max()/1000)
# ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(f)', loc='center', fontsize=20)
im = ax.imshow(torch.std(VAE_v_samp100, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp1, axis=0).max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(g)', loc='center', fontsize=20)
im = ax.imshow(torch.std(VAE_v_samp05, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp1, axis=0).max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

ax = fig.add_subplot(gs[1, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(h)', loc='center', fontsize=20)
im = ax.imshow(torch.std(VAE_v_samp01, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp1, axis=0).max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

#-------------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[2, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(i)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(VAE_v_samp1, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(VAE_v_samp1, axis=0)).max()/3/1000)
# ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 

# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(j)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(VAE_v_samp100, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(VAE_v_samp1, axis=0)).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(k)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(VAE_v_samp05, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(VAE_v_samp1, axis=0)).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(l)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(VAE_v_samp01, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(VAE_v_samp1, axis=0)).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.03,  
                    hspace=0.03)
# plt.show()
plt.savefig('./Figures/VAE-uncertainty-results-Ns.png', dpi=300, bbox_inches='tight')

In [ ]:
v_pre_3000 = torch.load('./Figures/VAE-G-inversion-results/v_pred_test/v_pred_{}.pth'.format(3000))


In [ ]:
v_pre_3000.shape


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE100 = torch.mean(VAE_v_samp100, axis=0)
v_samp_std_VAE100 = torch.std(VAE_v_samp100, axis=0)
v_samp_mean_VAE1 = torch.mean(VAE_v_samp1, axis=0)
v_samp_std_VAE1 = torch.std(VAE_v_samp1, axis=0)
v_samp_mean_VAE2 = torch.mean(VAE_v_samp05, axis=0)
v_samp_std_VAE2 = torch.std(VAE_v_samp05, axis=0)
v_samp_mean_VAE3 = torch.mean(VAE_v_samp01, axis=0)
v_samp_std_VAE3 = torch.std(VAE_v_samp01, axis=0)
nx=150
#模拟创建数据
normal_distribution_VAE100 = torch.distributions.Normal(v_samp_mean_VAE100[:,nx], v_samp_std_VAE1[:,nx])
samples_VAE100 = normal_distribution_VAE100.sample((500,))
normal_distribution_VAE1 = torch.distributions.Normal(v_samp_mean_VAE1[:,nx], v_samp_std_VAE1[:,nx])
samples_VAE1 = normal_distribution_VAE1.sample((500,))
normal_distribution_VAE2 = torch.distributions.Normal(v_samp_mean_VAE2[:,nx], v_samp_std_VAE2[:,nx])
samples_VAE2 = normal_distribution_VAE2.sample((500,))
normal_distribution_VAE3 = torch.distributions.Normal(v_samp_mean_VAE3[:,nx], v_samp_std_VAE3[:,nx])
samples_VAE3 = normal_distribution_VAE3.sample((500,))

data100 = samples_VAE100.T.numpy()
data1 = samples_VAE1.T.numpy()
data2 = samples_VAE2.T.numpy()
data3 = samples_VAE3.T.numpy()

#计算直方图
hist_data100 = []
bins = np.linspace(1500,5500,500)  #设定统一的bins
for i in range(data100.shape[0]):
    hist100, _ = np.histogram(data100[i], bins)  # 计算直方图
    hist_data100.append(hist100)
hist_data100 = np.array(hist_data100)

hist_data1 = []
bins = np.linspace(1500,5500,500)  #设定统一的bins
for i in range(data1.shape[0]):
    hist1, _ = np.histogram(data1[i], bins)  # 计算直方图
    hist_data1.append(hist1)
hist_data1 = np.array(hist_data1)

hist_data2 = []
bins2 = np.linspace(1500,5500,500)  #设定统一的bins
for i in range(data1.shape[0]):
    hist2, _ = np.histogram(data2[i], bins2)  # 计算直方图
    hist_data2.append(hist2)
hist_data2 = np.array(hist_data2)

hist_data3 = []
for i in range(data1.shape[0]):
    hist3, _ = np.histogram(data3[i], bins)  # 计算直方图
    hist_data3.append(hist3)
hist_data3 = np.array(hist_data3)


x = torch.arange(0, 128, dtype=torch.float32)*15/1000

cmap = 'RdBu_r'
fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(1, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=14)
ax.set_xlabel('Velocity [km/s]', fontsize=14)
ax.set_title('(a)', loc='center', fontsize=18)
im = ax.imshow(hist_data1/5, aspect='auto', vmax=1, vmin=0, extent=[1.43, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_VAE1[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='VAE')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='True')
# ax.plot(v_pre_3000[0, :,nx].cpu().detach().numpy()/1000, x, color='yellow', label='v_True')

ax.axhline(y=0.45, color='white', linewidth=1, xmin=0, xmax=1)  #30,60,90,120
ax.axhline(y=0.9, color='white', linewidth=1, xmin=0, xmax=1)  
ax.axhline(y=1.35, color='white', linewidth=1, xmin=0, xmax=1)  
ax.axhline(y=1.8, color='white', linewidth=1, xmin=0, xmax=1)  

# plt.legend(fontsize=12)
# plt.legend(loc=(0.64, 0.885))
# ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.show()

ax = fig.add_subplot(gs[0, 1])
# ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Velocity [km/s]', fontsize=14)
ax.set_title('(b)', loc='center', fontsize=18)
im = ax.imshow(hist_data100/5, aspect='auto', vmax=1, vmin=0, extent=[1.43, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_VAE100[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='vp_VAE')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='vp_True')
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.show()


ax = fig.add_subplot(gs[0, 2])
# ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Velocity [km/s]', fontsize=14)
ax.set_title('(c)', loc='center', fontsize=18)
im = ax.imshow(hist_data2/5, aspect='auto', vmax=1, vmin=0, extent=[1.42, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_VAE2[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='vp_VAE')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='vp_True')
# ax.legend(loc='best')
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.show()


ax = fig.add_subplot(gs[0, 3])
# ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Velocity [km/s]', fontsize=14)
ax.set_title('(d)', loc='center', fontsize=18)
im = ax.imshow(hist_data3/5, aspect='auto', vmax=1, vmin=0, extent=[1.43, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_VAE3[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='VAE')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='True')
plt.legend(fontsize=12)
# plt.legend(loc=(0.64, 0.885))
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="6%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('Probability', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.15)
plt.savefig('./Figures/figures/VAE-uncertainty-vprofile-Ns.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
v_samp_mean_VAE100 = torch.mean(VAE_v_samp100, axis=0)
v_samp_std_VAE100 = torch.std(VAE_v_samp100, axis=0)
v_samp_mean_VAE1 = torch.mean(VAE_v_samp1, axis=0)
v_samp_std_VAE1 = torch.std(VAE_v_samp1, axis=0)
v_samp_mean_VAE2 = torch.mean(VAE_v_samp05, axis=0)
v_samp_std_VAE2 = torch.std(VAE_v_samp05, axis=0)
v_samp_mean_VAE3 = torch.mean(VAE_v_samp01, axis=0)
v_samp_std_VAE3 = torch.std(VAE_v_samp01, axis=0)

positions = [30, 60, 90, 120]
std_VAE1 = torch.zeros((4, 1))
mean_VAE1 = torch.zeros((4, 1))
std_VAE100 = torch.zeros((4, 1))
mean_VAE100 = torch.zeros((4, 1))
std_VAE2 = torch.zeros((4, 1))
mean_VAE2 = torch.zeros((4, 1))
std_VAE3 = torch.zeros((4, 1))
mean_VAE3 = torch.zeros((4, 1))
for s in range(4):
    po = positions[s]
    std_VAE1[s] = v_samp_std_VAE1[po,150]/1000
    mean_VAE1[s] = v_samp_mean_VAE1[po,150]/1000
    std_VAE100[s] = v_samp_std_VAE100[po,150]/1000
    mean_VAE100[s] = v_samp_mean_VAE100[po,150]/1000
    std_VAE2[s] = v_samp_std_VAE2[po,150]/1000
    mean_VAE2[s] = v_samp_mean_VAE2[po,150]/1000
    std_VAE3[s] = v_samp_std_VAE3[po,150]/1000
    mean_VAE3[s] = v_samp_mean_VAE3[po,150]/1000

In [ ]:
VAE_v_samp1 = torch.load('./Figures/v_samp/v_samp_eval_VAE_N1.pth').cpu()
Drop_v_samp1 = torch.load('./Figures/v_samp/v_samp_train_Dropout_p0.1.pth').cpu()
AE_v_pred = torch.load('./Figures/v_samp/v_pred_AE_3000_copyfromold.pth')

vp_tensor = vp_tensor.cpu()
vi_tensor_true = vi_tensor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


v_samp_mean_VAE1 = torch.mean(VAE_v_samp1, axis=0)
v_samp_std_VAE1 = torch.std(VAE_v_samp1, axis=0)
v_samp_mean_Drop1 = torch.mean(Drop_v_samp1, axis=0)
v_samp_std_Drop1 = torch.std(Drop_v_samp1, axis=0)

nx1=65

#模拟创建数据
normal_distribution_VAE1 = torch.distributions.Normal(v_samp_mean_VAE1[:,nx1], v_samp_std_VAE1[:,nx1])
samples_VAE1 = normal_distribution_VAE1.sample((500,))
data_VAE1 = samples_VAE1.T.numpy()

normal_distribution_Drop1 = torch.distributions.Normal(v_samp_mean_Drop1[:,nx1], v_samp_std_Drop1[:,nx1])
samples_Drop1 = normal_distribution_Drop1.sample((500,))
data_Drop1 = samples_Drop1.T.numpy()

#计算直方图
hist_data1 = []
bins = np.linspace(1500,5500,500)  #设定统一的bins
for i in range(data_VAE1.shape[0]):
    hist1, _ = np.histogram(data_VAE1[i], bins)  # 计算直方图
    hist_data1.append(hist1)
hist_data1 = np.array(hist_data1)

hist_data2 = []
for i in range(data_Drop1.shape[0]):
    hist2, _ = np.histogram(data_Drop1[i], bins)  # 计算直方图
    hist_data2.append(hist2)
hist_data2 = np.array(hist_data2)

x = torch.arange(0, 128, dtype=torch.float32)*15/1000

cmap = 'RdBu_r'
fig = plt.figure(figsize=(8, 9))
gs = fig.add_gridspec(1, 2)
ax = fig.add_subplot(gs[0, 1])
# ax.set_ylabel('Depth [km]', fontsize=14)
ax.set_xlabel('Velocity [km/s]', fontsize=14)
ax.set_title('(b)', loc='center', fontsize=18)
im = ax.imshow(hist_data1/5, aspect='auto', vmax=1, vmin=0, extent=[1.45, 5.52, 1.92, 0]) 
ax.plot(AE_v_pred[0, :,nx1].cpu().detach().numpy()/1000, x, color='lavenderblush', linewidth=1.8, label='AE')
ax.plot(v_samp_mean_VAE1[:,nx1].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='VAE')
ax.plot(vi_tensor_true[0, :,nx1].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx1].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='True')

plt.legend(fontsize=12)
# plt.legend(loc=(0.608, 0.885))
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 

cbar = add_colorbar(ax, im, ax.transAxes, width="6%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Probability', size=14)
cbar = add_colorbar(ax, im, ax.transAxes, width="6%")
cbar.ax.tick_params(labelsize=12)  



ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=14)
ax.set_xlabel('Velocity [km/s]', fontsize=14)
ax.set_title('(a)', loc='center', fontsize=18)
im = ax.imshow(hist_data2/5, aspect='auto', vmax=1, vmin=0, extent=[1.45, 5.52, 1.92, 0]) 
ax.plot(AE_v_pred[0, :,nx1].cpu().detach().numpy()/1000, x, color='lavenderblush', linewidth=1.8, label='AE')
ax.plot(v_samp_mean_Drop1[:,nx1].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='Dropout')
ax.plot(vi_tensor_true[0, :,nx1].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx1].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='True')
plt.legend(fontsize=12)
# ax.set_yticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# plt.show()


# ax.legend(loc='best')
# ax.set_yticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.xticks(range(2, 6, 1))  # 设置刻度间隔为2
 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.03,  
                    hspace=0.15)

plt.savefig('./Figures/figures/compare-uncertainty-vprofile.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import pandas as pd
from scipy.ndimage import gaussian_filter

# Load the velocity model 
vmodel = np.array(pd.read_csv("./vel_marmousi_376x1151.csv")) 
# v_init = np.array(pd.read_csv("./vel_marmousi_smooth400_376x1151.csv"))
v_init = np.array(pd.read_csv("./vel_marmousi_376x1151.csv"))
# v_init = gaussian_filter(v_init, sigma=10)
v_init = gaussian_filter(v_init, sigma=15)

dz = 8
nz, nx = vmodel.shape
print("Original Model Shape: {}, Grid Interval: {}m".format(vmodel.shape, dz))

################# Plot true & initial velocity model #################
fig = plt.figure(figsize=(12, 2))
gs = fig.add_gridspec(1, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]")
im = ax.imshow(vmodel/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

ax = fig.add_subplot(gs[0, 1])
ax.set( xlabel="Distance x[km]")
im = ax.imshow(v_init/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
################# Plot true & initial velocity model #################


dz = 32
vp_tensor_init = torch.from_numpy(vmodel).type(dtype=torch.float32).to(device)
vi_tensor_init = torch.from_numpy(v_init).type(dtype=torch.float32).to(device)
sea = (torch.ones([9,1151])*1500).type(dtype=torch.float32).to(device)
vp_tensor = torch.cat([sea, vp_tensor_init], axis=0)[None, ::4, 70:1094:4]
vi_tensor_init = torch.from_numpy(gaussian_filter(vp_tensor_init.cpu().numpy(), sigma=30)).type(dtype=torch.float32).to(device)
vi_tensor = torch.cat([sea, vi_tensor_init], axis=0)[None, ::4, 70:1094:4]

nv, nz, nx = vp_tensor.shape
print("Resampled Model Shape: {}, Grid Interval: {}m".format((nz, nx), dz))

# Setting locations of sources and receivers
xs = torch.arange(10, nx-10, 12, dtype=torch.long).repeat([nv, 1])      # x-coordinate for sources
ns = xs.shape[1]                                                        # number of shots 
xr = torch.arange(0, nx, 1, dtype=torch.long).repeat([nv, ns, 1])       # x-coordinate for receivers
zs = torch.full((nv, ns), 0, dtype=torch.long)                          # depth of sources    震源深度
zr = torch.full((nv, ns, nx), 0, dtype=torch.long)                      # depth of receivers 地表
print("Number of shots: {}, with interval: {}m, in depth: {}m".format(ns, (xs[0, 1]-xs[0, 0])*dz, zs[0, 0]*dz))
print(xs)

################# Plot true & initial velocity model #################
fig = plt.figure(figsize=(12, 2))
gs = fig.add_gridspec(1, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]")
im = ax.imshow(vp_tensor[0].cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

ax = fig.add_subplot(gs[0, 1])
ax.set( xlabel="Distance x[km]")
im = ax.imshow(vi_tensor[0].cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# ################# Plot true & initial velocity model #################

In [ ]:
VAE_v_samp = torch.load('./Figures/v_samp/v_samp_eval_VAE_scale.pth').cpu()
vi_tensor = torch.load('./Figures/v_samp/vi_tensor_scale.pth').cpu()
vp_tensor = torch.load('./Figures/v_samp/vp_tensor_scale.pth').cpu()

VAE_v_samp2 = torch.load('./Figures/v_samp/v_samp_add/v_samp_VAE_over_scale4Hz.pth').cpu()
vi_tensor2 = torch.load('./Figures/v_samp/vi_tensor_over.pth').cpu()
vp_tensor2 = torch.load('./Figures/v_samp/vp_tensor_over.pth').cpu()

In [ ]:
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FormatStrFormatter

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 6))
gs = fig.add_gridspec(2, 5)

nv, nz, nx = vp_tensor.shape
dz = 32

ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=15)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(vp_tensor[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
ax.tick_params(axis='y', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(vi_tensor[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(torch.std(VAE_v_samp, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp, axis=0).max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[0, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(VAE_v_samp, axis=0)).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp, axis=0).max()/2/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
# cbar.set_label('km/s', size=14) 
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

##------------------------------------------------------------------------------------------------------------------
nv, nz, nx = vp_tensor2.shape
dz = 25

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=15)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(vp_tensor2[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor2.min()/1000, vmax=vp_tensor2.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
ax.tick_params(axis='y', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(vi_tensor2[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor2.min()/1000, vmax=vp_tensor2.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp2, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor2.min()/1000, vmax=vp_tensor2.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[1, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(torch.std(VAE_v_samp2, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(VAE_v_samp2, axis=0).max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 
# 设置颜色条的刻度保留一位小数
cbar.formatter = FormatStrFormatter('%1.1f')
cbar.update_ticks()

ax = fig.add_subplot(gs[1, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor2[0]-torch.mean(VAE_v_samp2, axis=0)).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
              vmin=0, vmax=torch.std(VAE_v_samp2, axis=0).max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
# cbar.set_label('km/s', size=14) 
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 



plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.12,  
                    hspace=0.3)

# plt.savefig('./Figures/figures/compare-model-inversion-results.png', dpi=300, bbox_inches='tight')


In [ ]:
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FormatStrFormatter

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 6))
gs = fig.add_gridspec(2, 5)

nv, nz, nx = vp_tensor.shape
dz = 32

ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=15)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(vi_tensor[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
ax.tick_params(axis='y', labelsize=13) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# cbar.ax.tick_params(labelsize=13)  
# cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(vp_tensor[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# cbar.ax.tick_params(labelsize=13)  
# cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

# ax = fig.add_subplot(gs[0, 3])
# ax.set_ylabel('Depth (km)', fontsize=15)
# ax.set_xlabel('Distance [km]', fontsize=15)
# # ax.set_title('(d)', loc='center', fontsize=20)
# im = ax.imshow(torch.std(VAE_v_samp, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
#                vmin=0, vmax=torch.std(VAE_v_samp, axis=0).max()/1000)
# # ax.set_yticks([])
# # ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=13) 
# ax.tick_params(axis='y', labelsize=13) 
# # cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# # cbar.ax.tick_params(labelsize=13)  
# # cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

# ax = fig.add_subplot(gs[0, 4])
# #ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=15)
# # ax.set_title('(e)', loc='center', fontsize=20)
# im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(VAE_v_samp, axis=0)).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1.33, cmap=cmap,
#                vmin=0, vmax=torch.std(VAE_v_samp, axis=0).max()/1000)
# ax.set_yticks([])
# # ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=13) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# cbar.ax.tick_params(labelsize=13)  
# # cbar.set_label('km/s', size=14) 
# cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

##------------------------------------------------------------------------------------------------------------------
nv, nz, nx = vp_tensor2.shape
dz = 25

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=15)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(vi_tensor2[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor2.min()/1000, vmax=vp_tensor2.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
ax.tick_params(axis='y', labelsize=13) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# cbar.ax.tick_params(labelsize=13)  
# cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(vp_tensor2[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor2.min()/1000, vmax=vp_tensor2.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
# cbar.ax.tick_params(labelsize=13)  
# cbar.ax.set_title('km/s', fontsize=14, pad=-35) 

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=15)
# ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(VAE_v_samp2, axis=0).detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor2.min()/1000, vmax=vp_tensor2.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=13) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=13)  
cbar.ax.set_title('km/s', fontsize=14, pad=-35) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=-0.3,  
                    hspace=0.3)

# plt.savefig('./Figures/figures/compare-model-inversion-results-new1.png', dpi=300, bbox_inches='tight')


In [ ]:
v_samp_mean_VAE100 = torch.mean(VAE_v_samp100, axis=0)
v_samp_std_VAE100 = torch.std(VAE_v_samp100, axis=0)
v_samp_mean_VAE1 = torch.mean(VAE_v_samp1, axis=0)
v_samp_std_VAE1 = torch.std(VAE_v_samp1, axis=0)
v_samp_mean_VAE2 = torch.mean(VAE_v_samp05, axis=0)
v_samp_std_VAE2 = torch.std(VAE_v_samp05, axis=0)
v_samp_mean_VAE3 = torch.mean(VAE_v_samp01, axis=0)
v_samp_std_VAE3 = torch.std(VAE_v_samp01, axis=0)

positions = [30, 60, 90, 120]
std_VAE1 = torch.zeros((4, 1))
mean_VAE1 = torch.zeros((4, 1))
std_VAE100 = torch.zeros((4, 1))
mean_VAE100 = torch.zeros((4, 1))
std_VAE2 = torch.zeros((4, 1))
mean_VAE2 = torch.zeros((4, 1))
std_VAE3 = torch.zeros((4, 1))
mean_VAE3 = torch.zeros((4, 1))
for s in range(4):
    po = positions[s]
    std_VAE1[s] = v_samp_std_VAE1[po,150]/1000
    mean_VAE1[s] = v_samp_mean_VAE1[po,150]/1000
    std_VAE100[s] = v_samp_std_VAE100[po,150]/1000
    mean_VAE100[s] = v_samp_mean_VAE100[po,150]/1000
    std_VAE2[s] = v_samp_std_VAE2[po,150]/1000
    mean_VAE2[s] = v_samp_mean_VAE2[po,150]/1000
    std_VAE3[s] = v_samp_std_VAE3[po,150]/1000
    mean_VAE3[s] = v_samp_mean_VAE3[po,150]/1000

In [ ]:
from scipy.stats import norm
from matplotlib.ticker import ScalarFormatter

zz = 30
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.06
z = np.linspace(1500, 5500, 3000)/1000
fig = plt.figure(figsize=(4, 10))
gs = fig.add_gridspec(4, 1)
ax = fig.add_subplot(gs[0, 0])
pdf1 = norm.pdf(z, mean_VAE1[0].cpu().detach().numpy(), std_VAE1[0].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[0].cpu().detach().numpy(), std_VAE2[0].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[0].cpu().detach().numpy(), std_VAE3[0].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[0].cpu().detach().numpy(), std_VAE100[0].cpu().detach().numpy())

ax.set_title('(i)', loc='center', fontsize=20)
ax.plot(z, pdf1, label='N(0,1)', linewidth=2, color='orange')
ax.plot(z, pdf4, label='N(0,100)', linewidth=2, color='dodgerblue')
ax.plot(z, pdf2, label='N(0,0.25)', linewidth=2, color='purple')
ax.plot(z, pdf3, label='N(0,0.01)', linewidth=2, color='green')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# ax.set_xlabel("Velocity [Km/s]", fontsize=16)
# ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.set_ylabel("PDF", fontsize=16)
plt.legend(fontsize=12, ncol=1, loc='upper right')
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

zz = 60
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.03
ax = fig.add_subplot(gs[1, 0])
pdf1 = norm.pdf(z, mean_VAE1[1].cpu().detach().numpy(), std_VAE1[1].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[1].cpu().detach().numpy(), std_VAE2[1].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[1].cpu().detach().numpy(), std_VAE3[1].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[1].cpu().detach().numpy(), std_VAE100[1].cpu().detach().numpy())

ax.set_title('(ii)', loc='center', fontsize=20)
ax.plot(z, pdf1, label='N(0,1)', linewidth=2, color='orange')
ax.plot(z, pdf4, label='N(0,100)', linewidth=2, color='dodgerblue')
ax.plot(z, pdf2, label='N(0,0.25)', linewidth=2, color='purple')
ax.plot(z, pdf3, label='N(0,0.01)', linewidth=2, color='green')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# ax.set_xlabel("Velocity [Km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
# plt.legend(fontsize=12, ncol=1, loc='upper right')
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)


zz = 90
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.02
ax = fig.add_subplot(gs[2, 0])
pdf1 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE1[2].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[2].cpu().detach().numpy(), std_VAE2[2].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE3[2].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[2].cpu().detach().numpy(), std_VAE100[2].cpu().detach().numpy())

ax.set_title('(iii)', loc='center', fontsize=20)
ax.plot(z, pdf1, label='N(0,1)', linewidth=2, color='orange')
ax.plot(z, pdf4, label='N(0,100)', linewidth=2, color='dodgerblue')
ax.plot(z, pdf2, label='N(0,0.25)', linewidth=2, color='purple')
ax.plot(z, pdf3, label='N(0,0.01)', linewidth=2, color='green')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
# ax.set_xlabel("Velocity [Km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# plt.legend(fontsize=12, ncol=1)
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)


zz = 120
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.03
ax = fig.add_subplot(gs[3, 0])
pdf1 = norm.pdf(z, mean_VAE1[3].cpu().detach().numpy(), std_VAE1[3].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[3].cpu().detach().numpy(), std_VAE2[3].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[3].cpu().detach().numpy(), std_VAE3[3].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[3].cpu().detach().numpy(), std_VAE100[3].cpu().detach().numpy())

ax.set_title('(iv)', loc='center', fontsize=20)
ax.plot(z, pdf1, label='N(0,1)', linewidth=2, color='orange')
ax.plot(z, pdf4, label='N(0,100)', linewidth=2, color='dodgerblue')
ax.plot(z, pdf2, label='N(0,0.25)', linewidth=2, color='purple')
ax.plot(z, pdf3, label='N(0,0.01)', linewidth=2, color='green')
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True')
ax.set_xlabel("Velocity [km/s]", fontsize=17)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# plt.legend(fontsize=12, ncol=1)

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1,  
                    hspace=0.4)

plt.savefig('./Figures/figures/VAE-uncertainty-point-dis-peitu.png', dpi=300, bbox_inches='tight')


In [ ]:
v_samp_mean_VAE100 = torch.mean(VAE_v_samp100, axis=0)
v_samp_std_VAE100 = torch.std(VAE_v_samp100, axis=0)
v_samp_mean_VAE1 = torch.mean(VAE_v_samp1, axis=0)
v_samp_std_VAE1 = torch.std(VAE_v_samp1, axis=0)
v_samp_mean_VAE2 = torch.mean(VAE_v_samp05, axis=0)
v_samp_std_VAE2 = torch.std(VAE_v_samp05, axis=0)
v_samp_mean_VAE3 = torch.mean(VAE_v_samp01, axis=0)
v_samp_std_VAE3 = torch.std(VAE_v_samp01, axis=0)

In [ ]:
train_loss_history_N1 = torch.load('./Figures/train_loss_history/VAE_train_loss_history_N1.pth')
train_loss_history_N01 = torch.load('./Figures/train_loss_history/VAE_train_loss_history_N01.pth')
train_loss_history_N05 = torch.load('./Figures/train_loss_history/VAE_train_loss_history_N05.pth')
train_loss_history_N100 = torch.load('./Figures/train_loss_history/VAE_train_loss_history_N10.pth')
train_loss_history_N1 = torch.tensor(train_loss_history_N1)
train_loss_history_N01 = torch.tensor(train_loss_history_N01)
train_loss_history_N05 = torch.tensor(train_loss_history_N05)
train_loss_history_N100 = torch.tensor(train_loss_history_N100)


In [ ]:
n=3000
z = torch.arange(0, n)

fig = plt.figure(figsize=(28, 3.5))
gs = fig.add_gridspec(1, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, np.log(train_loss_history_N1[0, 0:n]/1), color='red', linestyle='-', linewidth=1.5, label='N(0,1)')
ax.plot(z, np.log(train_loss_history_N01[0, 0:n]/1), color='dodgerblue', linestyle='-', linewidth=1.5, label='N(0,0.01)')
ax.plot(z, np.log(train_loss_history_N05[0, 0:n]*1), color='purple', linestyle='-', linewidth=1.5, label='N(0,0.25)')
# ax.plot(z, train_loss_history_N001[0, 0:n]/10000, color='orange', linestyle='-', linewidth=1.5, label='N0.01')
ax.plot(z, np.log(train_loss_history_N100[0, 0:n]*1), color='green', linestyle='-', linewidth=1.5, label='N(0,100)')
# ax.plot(z, train_loss_history[0], color='red', linestyle='-', linewidth=1.5, label='train_loss')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss all (log scale)", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.legend(fontsize=12, ncol=2, loc='upper right')
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.set_ylim(-0.2*1e20, 4*1e20)

ax = fig.add_subplot(gs[0, 1])
ax.set_title('(b)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_N1[1, 0:n], color='red', linestyle='-', linewidth=1.5, label='N(0,1)')
ax.plot(z, train_loss_history_N01[1, 0:n], color='dodgerblue', linestyle='-', linewidth=1.5, label='N(0,0.01)')
ax.plot(z, train_loss_history_N05[1, 0:n], color='purple', linestyle='-', linewidth=1.5, label='N(0,0.25)')
# ax.plot(z, train_loss_history_N001[1, 0:n], color='orange', linestyle='-', linewidth=1.5, label='N0.01')
ax.plot(z, train_loss_history_N100[1, 0:n], color='green', linestyle='-', linewidth=1.5, label='N(0,100)')
# ax.plot(z, train_loss_history[1, 0:n], color='red', linestyle='-', linewidth=1.5, label='train_loss')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss pixel", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.legend(fontsize=12, ncol=2)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[0, 2])
ax.set_title('(c)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_N1[2, 0:n], color='red', linestyle='-', linewidth=1.5, label='N(0,1)')
ax.plot(z, train_loss_history_N01[2, 0:n], color='dodgerblue', linestyle='-', linewidth=1.5, label='N(0,0.01)')
ax.plot(z, train_loss_history_N05[2, 0:n], color='purple', linestyle='-', linewidth=1.5, label='N(0,0.25)')
# ax.plot(z, train_loss_history_N001[2, 0:n], color='orange', linestyle='-', linewidth=1.5, label='N0.01')
ax.plot(z, train_loss_history_N100[2, 0:n], color='green', linestyle='-', linewidth=1.5, label='N(0,100)')
# ax.plot(z, train_loss_history[2, 0:n], color='red', linestyle='-', linewidth=1.5, label='train_loss')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss perceptual", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.legend(fontsize=12, ncol=2)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[0, 3])
ax.set_title('(d)', loc='center', fontsize=20)
ax.plot(z, np.log(train_loss_history_N1[3, 0:n]/1), color='red', linestyle='-', linewidth=1.5, label='N(0,1)')
ax.plot(z, np.log(train_loss_history_N01[3, 0:n]/1), color='dodgerblue', linestyle='-', linewidth=1.5, label='N(0,0.01)')
ax.plot(z, np.log(train_loss_history_N05[3, 0:n]*1), color='purple', linestyle='-', linewidth=1.5, label='N(0,0.25)')
# ax.plot(z, train_loss_history_N001[3, 0:n]/10000, color='orange', linestyle='-', linewidth=1.5, label='N0.01')
ax.plot(z, np.log(train_loss_history_N100[3, 0:n]*1), color='green', linestyle='-', linewidth=1.5, label='N(0,100)')
# ax.plot(z, train_loss_history[2], color='red', linestyle='-', linewidth=1.5, label='train_loss')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss KLD (log scale)", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.legend(fontsize=12, ncol=2, loc='upper right')
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.set_ylim(-0.2*1e20, 4*1e20)

plt.savefig('./Figures/figures/VAE-loss.png', dpi=300, bbox_inches='tight')


In [ ]:
# VAE 汇总展示 Z
# epochs = [1, 200, 200, 500, 2900]
epochs = [1, 200, 500, 1000, 3000]
Z_mean_G = torch.zeros((5, 256))
Z_std_G = torch.zeros((5, 256))
Z_mean_GN100 = torch.zeros((5, 256))
Z_std_GN100 = torch.zeros((5, 256))
Z_mean_GN05 = torch.zeros((5, 256))
Z_std_GN05 = torch.zeros((5, 256))
Z_mean_GN01 = torch.zeros((5, 256))
Z_std_GN01 = torch.zeros((5, 256))

for s in range(5):
    epoch = epochs[s]
    Z_mean_G[s] = torch.load('./Figures/ZVAE/Z_mean_VAE_N1/Z_mean_epoch_{}'.format(epoch))
    Z_std_G[s]= torch.load('./Figures/ZVAE/Z_std_VAE_N1/Z_std_epoch_{}'.format(epoch))
    Z_mean_GN100[s] = torch.load('./Figures/ZVAE/Z_mean_VAE_N10/Z_mean_epoch_{}'.format(epoch))
    Z_std_GN100[s] = torch.load('./Figures/ZVAE/Z_std_VAE_N10/Z_std_epoch_{}'.format(epoch))
    Z_mean_GN05[s] = torch.load('./Figures/ZVAE/Z_mean_VAE_N05/Z_mean5_epoch_{}'.format(epoch))
    Z_std_GN05[s] = torch.load('./Figures/ZVAE/Z_std_VAE_N05/Z_std5_epoch_{}'.format(epoch))
    Z_mean_GN01[s] = torch.load('./Figures/ZVAE/Z_mean_VAE_N01/Z_mean_epoch_{}'.format(epoch))
    Z_std_GN01[s] = torch.load('./Figures/ZVAE/Z_std_VAE_N01/Z_std_epoch_{}'.format(epoch))
    
    

In [ ]:
from scipy.stats import norm

# Initialize figure and grid layout
fig = plt.figure(figsize=(26, 10))  # Adjusted figure size for 3 rows
gs = fig.add_gridspec(4, 5)  # Now using 3 rows and 5 columns

# Titles for the columns
# titles = ['1 epoch', '100 epochs', '200 epochs', '500 epochs', '3000 epochs']
titles = ['(a)', '(b)', '(c)', '(d)', '(e)']

# Plot each distribution in its designated row
for s in range(5):  # Loop over columns
    for r, color, label in zip(range(4), ['blue', 'green', 'pink', 'orange'], ['N(0,1)', 'N(0,100)', 'N(0,0.25)', 'N(0,0.01)']):
        # Define the epoch and title
        epoch = epochs[s]
        title = titles[s]

        # Create the subplot at the given row and column
        ax = fig.add_subplot(gs[r, s])

        # Plot each distribution line
        for i in range(256):
            # Grab the means and standard deviations for the given distribution
            if r == 0:
                Z_mean = Z_mean_G[s, i]
                Z_std = Z_std_G[s, i]
                z = np.linspace(-5, 5, 3000)
            elif r == 1:
                if s==0:
                    Z_mean = Z_mean_GN100[s, i]
                    Z_std = Z_std_GN100[s, i]
                    z = np.linspace(-5, 5, 3000)
                else:
                    Z_mean = Z_mean_GN100[s, i]
                    Z_std = Z_std_GN100[s, i]
                    z = np.linspace(-50, 50, 3000)
            elif r == 2:
                Z_mean = Z_mean_GN05[s, i]
                Z_std = Z_std_GN05[s, i]
                z = np.linspace(-5, 5, 3000)
            elif r == 3:
                Z_mean = Z_mean_GN01[s, i]
                Z_std = Z_std_GN01[s, i]
                z = np.linspace(-5, 5, 3000)

            # Compute the normal distribution's probability density function
            pdf = norm.pdf(z, Z_mean.cpu().detach().numpy(), Z_std.cpu().detach().numpy())

            if i == 0: # 设置只在第一个循环中添加label
                ax.plot(z, pdf, label=label, linewidth=2, color='red')
                ax.tick_params(axis='x', labelsize=12) 
                ax.tick_params(axis='y', labelsize=12) 
            else:
                ax.plot(z, pdf, linewidth=2, color=color)
                ax.tick_params(axis='x', labelsize=12) 
                ax.tick_params(axis='y', labelsize=12) 
        # Plot the red line in each subplot for the prior distribution
        if r == 0:
            pdf_Z = norm.pdf(z, 0, 1)
            ax.plot(z, pdf_Z, linewidth=2, color='red')
            ax.tick_params(axis='x', labelsize=12) 
            ax.tick_params(axis='y', labelsize=12) 
        elif r == 1:
            pdf_ZN100 = norm.pdf(z, 0, 10)
            ax.plot(z, pdf_ZN100, linewidth=2, color='red')
            ax.tick_params(axis='x', labelsize=12) 
            ax.tick_params(axis='y', labelsize=12) 
        elif r == 2:
            pdf_ZN05 = norm.pdf(z, 0, 0.5)
            ax.plot(z, pdf_ZN05, linewidth=2, color='red')
            ax.tick_params(axis='x', labelsize=12) 
            ax.tick_params(axis='y', labelsize=12) 
        elif r == 3:
            pdf_ZN01 = norm.pdf(z, 0, 0.1)
            ax.plot(z, pdf_ZN01, linewidth=2, color='red')
            ax.tick_params(axis='x', labelsize=12) 
            ax.tick_params(axis='y', labelsize=12) 

        # Title adjustments
        if s == 0:
            ax.set_ylabel('PDF', fontsize=14)
        # if r == 0: 
            # ax.set_title(title, loc='center', fontsize=18)
        if s == 4: # 只在每一行的最后一列显示图例
            ax.legend(fontsize=12)

# Display the plot
plt.tight_layout()
plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.19,  
                    hspace=0.2)

plt.savefig('./Figures/figures/VAE-z-distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# AE 示意图汇总

In [ ]:
# AE 汇总展示反演过程的vpred
epochs = [1, 100, 500, 1000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/AE-G-inversion-results/v_pred_test/v_pred_{}.pth'.format(epoch))
# print(v_pred)  

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 3))
gs = fig.add_gridspec(1, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=14)
ax.set_xlabel('Distance [km]', fontsize=14)
ax.set_title('(a)', loc='center', fontsize=18)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=14)
ax.set_title('(b)', loc='center', fontsize=18)
im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=14)
ax.set_title('(c)', loc='center', fontsize=18)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.4)

plt.savefig('./Figures/uncertainty figures/AE-G-inversion-results-test.png', dpi=300, bbox_inches='tight')


In [ ]:
# AE 汇总展示反演过程的vpred
epochs = [1, 100, 500, 1000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/AE-G-inversion-results/v_pred_test/v_pred_{}.pth'.format(epoch))
# print(v_pred)  

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 3))
gs = fig.add_gridspec(1, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('km/s', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.4)

plt.savefig('./Figures/AE-G-inversion-results-test.png', dpi=300, bbox_inches='tight')


In [ ]:
# AE 汇总展示反演过程的vpred
epochs = [1, 100, 500, 1000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/AE-G-inversion-results/v_pred_test/v_pred_{}.pth'.format(epoch))
# print(v_pred)  


nz = 128
z = dz * torch.arange(0, nz, dtype=torch.float32)/1000                    
v_pred = v_pred/1000
fig = plt.figure(figsize=(22.5, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
nnx0=65
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx0].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1')
# ax.plot(z, v_pred[1, :, nnx0].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='Iter=100')
ax.plot(z, v_pred[2, :, nnx0].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='Iter=500')
# ax.plot(z, v_pred[3, :, nnx0].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1000')
ax.plot(z, v_pred[4, :, nnx0].detach().cpu().numpy(), color='purple', linestyle='-', linewidth=1.5, label='Iter=3000')
ax.plot(z, vp_tensor[0, :, nnx0].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='True')
ax.set_xlabel("Depth [km]", fontsize=16)
ax.set_ylabel("Velocity [km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

ax = fig.add_subplot(gs[0, 1])
nnx1=135
ax.set_title('(b)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx1].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1')
# ax.plot(z, v_pred[1, :, nnx1].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='Iter=100')
ax.plot(z, v_pred[2, :, nnx1].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='Iter=500')
# ax.plot(z, v_pred[3, :, nnx1].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1000')
ax.plot(z, v_pred[4, :, nnx1].detach().cpu().numpy(), color='purple', linestyle='-', linewidth=1.5, label='Iter=3000')
ax.plot(z, vp_tensor[0, :, nnx1].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='True')
ax.set_xlabel("Depth [km]", fontsize=16)
ax.set_ylabel("Velocity [km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

ax = fig.add_subplot(gs[0, 2])
nnx2=240
ax.set_title('(c)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx2].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1')
# ax.plot(z, v_pred[1, :, nnx2].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='Iter=100')
ax.plot(z, v_pred[2, :, nnx2].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='Iter=500')
# ax.plot(z, v_pred[3, :, nnx2].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1000')
ax.plot(z, v_pred[4, :, nnx2].detach().cpu().numpy(), color='purple', linestyle='-', linewidth=1.5, label='Iter=3000')
ax.plot(z, vp_tensor[0, :, nnx2].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='True')
ax.set_xlabel("Depth [km]", fontsize=16)
ax.set_ylabel("Velocity [km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()
# ax.grid(True)
plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.12,  
                    hspace=0.18)

plt.savefig('./Figures/uncertainty figures/AE-vprofile.png', dpi=300, bbox_inches='tight')

In [ ]:
# AE 汇总展示反演过程的vpred
epochs = [1, 100, 500, 1000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/AE-G-inversion-results/v_pred_test/v_pred_{}.pth'.format(epoch))
# print(v_pred)  


nz = 128
z = dz * torch.arange(0, nz, dtype=torch.float32)/1000                    
v_pred = v_pred/1000
fig = plt.figure(figsize=(22.5, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
nnx0=65
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx0].detach().cpu().numpy(), color='black', linestyle='-', linewidth=1.5, label='Iter=1')
ax.plot(z, v_pred[1, :, nnx0].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='Iter=100')
ax.plot(z, v_pred[2, :, nnx0].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='Iter=500')
ax.plot(z, v_pred[3, :, nnx0].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1000')
ax.plot(z, v_pred[4, :, nnx0].detach().cpu().numpy(), color='purple', linestyle='-', linewidth=1.5, label='Iter=3000')
ax.plot(z, vp_tensor[0, :, nnx0].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='True')
ax.set_xlabel("Depth [km]", fontsize=16)
ax.set_ylabel("Velocity [km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

ax = fig.add_subplot(gs[0, 1])
nnx1=135
ax.set_title('(b)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx1].detach().cpu().numpy(), color='black', linestyle='-', linewidth=1.5, label='Iter=1')
ax.plot(z, v_pred[1, :, nnx1].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='Iter=100')
ax.plot(z, v_pred[2, :, nnx1].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='Iter=500')
ax.plot(z, v_pred[3, :, nnx1].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1000')
ax.plot(z, v_pred[4, :, nnx1].detach().cpu().numpy(), color='purple', linestyle='-', linewidth=1.5, label='Iter=3000')
ax.plot(z, vp_tensor[0, :, nnx1].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='True')
ax.set_xlabel("Depth [km]", fontsize=16)
ax.set_ylabel("Velocity [km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

ax = fig.add_subplot(gs[0, 2])
nnx2=240
ax.set_title('(c)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx2].detach().cpu().numpy(), color='black', linestyle='-', linewidth=1.5, label='Iter=1')
ax.plot(z, v_pred[1, :, nnx2].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='Iter=100')
ax.plot(z, v_pred[2, :, nnx2].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='Iter=500')
ax.plot(z, v_pred[3, :, nnx2].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='Iter=1000')
ax.plot(z, v_pred[4, :, nnx2].detach().cpu().numpy(), color='purple', linestyle='-', linewidth=1.5, label='Iter=3000')
ax.plot(z, vp_tensor[0, :, nnx2].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='True')
ax.set_xlabel("Depth [km]", fontsize=16)
ax.set_ylabel("Velocity [km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()
# ax.grid(True)
plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.12,  
                    hspace=0.18)

plt.savefig('./Figures/uncertainty figures/AE-vprofile.png', dpi=300, bbox_inches='tight')

In [ ]:
data_STD3 = torch.load("./Figures/shots_marmousi_I_2048_STD0.3.npz").detach().cpu().numpy()
data_STD2 = torch.load("./Figures/shots_marmousi_I_2048_STD0.2.npz").detach().cpu().numpy()
data_STD1 = torch.load("./Figures/shots_marmousi_I_2048_STD0.1.npz").detach().cpu().numpy()
data = torch.load("./shots_marmousi_I_2048.npz").detach().cpu().numpy()

dz = 15
xr = torch.load('./Figures/xr.npz')
xs = torch.load('./Figures/xs.npz')
offset = (xr[0, 0] - xs[0, 6]) * dz / 1000

In [ ]:
fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(1, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Time [s]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow((data[0, 6]), extent=[offset.min(), offset.max(), 1.9456, 0], aspect=3.5, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# 设置colorbar的取值范围
vmin = -((data[0, 6])).max()/40  # 最小值
vmax = ((data[0, 6])).max()/40   # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[0, 1])
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(((data_STD1[0, 6])), extent=[offset.min(), offset.max(), 1.9456, 0], aspect=3.5, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.set_yticks([])
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# 设置colorbar的取值范围
vmin = -((data[0, 6])).max()/40  # 最小值
vmax = ((data[0, 6])).max()/40  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[0, 2])
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(((data_STD2[0, 6])), extent=[offset.min(), offset.max(), 1.9456, 0], aspect=3.5, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.set_yticks([])
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# 设置colorbar的取值范围
vmin = -((data[0, 6])).max()/40  # 最小值
vmax = ((data[0, 6])).max()/40  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[0, 3])
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(((data_STD3[0, 6])), extent=[offset.min(), offset.max(), 1.9456, 0], aspect=3.5, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.set_yticks([])
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# 设置colorbar的取值范围
vmin = -((data[0, 6])).max()/40  # 最小值
vmax = ((data[0, 6])).max()/40  # 最大值
im.set_clim(vmin, vmax)

# ax = fig.add_subplot(gs[0, 4])
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(e)', loc='center', fontsize=20)
# im = ax.imshow(((data_wolow5[0, 6])), extent=[0, nz*dz/1000, 1024, 0], aspect=0.004, cmap='RdBu_r')
# ax.tick_params(axis='x', labelsize=12) 
# ax.set_yticks([])
# # cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# # 设置colorbar的取值范围
# vmin = -((data[0, 6])).max()/40  # 最小值
# vmax = ((data[0, 6])).max()/40   # 最大值
# im.set_clim(vmin, vmax)
# # cbar.ax.tick_params(labelsize=12)  
# # cbar.set_label('Km/s', size=14) 

# ax = fig.add_subplot(gs[0, 5])
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(f)', loc='center', fontsize=20)
# im = ax.imshow(((data_wolow7[0, 6])), extent=[0, nz*dz/1000, 1024, 0], aspect=0.004, cmap='RdBu_r')
# ax.tick_params(axis='x', labelsize=12) 
# ax.set_yticks([])
# # cbar = add_colorbar(ax, im, ax.transAxes, width="4%")
# # 设置colorbar的取值范围
# vmin = -((data[0, 6])).max()/40  # 最小值
# vmax = ((data[0, 6])).max()/40   # 最大值
# im.set_clim(vmin, vmax)
# cbar.ax.tick_params(labelsize=12)  
# cbar.set_label('Km/s', size=16) 

# ax = fig.add_subplot(gs[0, 6])
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(g)', loc='center', fontsize=20)
# im = ax.imshow(((data_wolow7[0, 6])), extent=[0, nz*dz/1000, 1024, 0], aspect=0.004, cmap='RdBu_r')
# ax.tick_params(axis='x', labelsize=12) 
# ax.set_yticks([])
cbar = add_colorbar(ax, im, ax.transAxes, width="4%")
# 设置colorbar的取值范围
vmin = -((data[0, 6])).max()/40  # 最小值
vmax = ((data[0, 6])).max()/40   # 最大值
im.set_clim(vmin, vmax)
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 
# 假设你已经有一个colorbar实例cbar
position = cbar.ax.get_position()
cbar.ax.set_position([position.x0, position.y0, position.width, position.height])

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.028,  
                    hspace=0.4)

plt.savefig('./Figures/figures/shots.png', dpi=300, bbox_inches='tight')

In [ ]:
data_wolow5 = torch.load("./shots_wo_low5_marmousi_I_1024.npz")
data_wolow6 = torch.load("./shots_wo_low6_marmousi_I_1024.npz")
data_wolow7 = torch.load("./shots_wo_low7_marmousi_I_1024.npz")
data = torch.load("./shots_marmousi_I_2048.npz")


In [ ]:
n = 40
fmax = 20
data_fx =torch.fft.rfft(data[0,6], dim=-2).cpu().numpy()
data_wolow5_fx =torch.fft.rfft(data_wolow5[0,6], dim=-2).cpu().numpy()
data_wolow7_fx =torch.fft.rfft(data_wolow7[0,6], dim=-2).cpu().numpy()

fig = plt.figure(figsize=(12, 6))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Frequency [Hz]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow((data_fx[0:n, :].real), extent=[offset.min(), offset.max(), fmax, 0], aspect=0.15, cmap='RdBu_r')
# im = ax.imshow((data_fx[0:64, :].real), extent=[0, nz*dz/1000, 32, 0], aspect=0.05, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# 设置colorbar的取值范围
vmin = -(data_fx[0:n, :].real).max()/60  # 最小值
vmax = (data_fx[0:n, :].real).max()/60   # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[0, 1])
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow((data_wolow5_fx[0:n, :].real), extent=[offset.min(), offset.max(), fmax, 0], aspect=0.15, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.set_yticks([])
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')
# 设置colorbar的取值范围
vmin = -(data_fx[0:n, :].real).max()/60  # 最小值
vmax = (data_fx[0:n, :].real).max()/60   # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[0, 2])
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow((data_wolow7_fx[0:n, :].real), extent=[offset.min(), offset.max(), fmax, 0], aspect=0.15, cmap='RdBu_r')
ax.tick_params(axis='x', labelsize=14) 
ax.set_yticks([])
cbar = add_colorbar(ax, im, ax.transAxes, width="4%")
# 设置colorbar的取值范围
vmin = -(data_fx[0:n, :].real).max()/60  # 最小值
vmax = (data_fx[0:n, :].real).max()/60   # 最大值
im.set_clim(vmin, vmax)
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.4)

plt.savefig('./Figures/uncertainty figures/shots-frequency.png', dpi=300, bbox_inches='tight')

In [ ]:
n=64
z = torch.arange(0, n)/2
fig = plt.figure(figsize=(22, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
# ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, np.mean(np.abs(data_fx[0:n, :]), axis=1), color='red', linestyle='-', linewidth=1.5, label='Full')
ax.plot(z, np.mean(np.abs(data_wolow5_fx[0:n, :]), axis=1), color='dodgerblue', linestyle='-', linewidth=1.5, label='Wo5hz')
ax.plot(z, np.mean(np.abs(data_wolow7_fx[0:n, :]), axis=1), color='green', linestyle='-', linewidth=1.5, label='Wo7hz')
ax.set_xlabel("Frequency [Hz]", fontsize=16)
ax.set_ylabel("Amp", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=14, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

# plt.savefig('./Figures/uncertainty figures/shots-frequency-spectrum.png', dpi=300, bbox_inches='tight')

In [ ]:
v_pred_no3 = torch.load('./Figures/AE-G-inversion-results/v_pred_noised0.3.pth').detach().cpu().numpy()
v_pred_no2 = torch.load('./Figures/AE-G-inversion-results/v_pred_noised0.2.pth').detach().cpu().numpy()
v_pred_no1 = torch.load('./Figures/AE-G-inversion-results/v_pred_noised0.1.pth').detach().cpu().numpy()

v_pred_wo5 = torch.load('./Figures/AE-G-inversion-results/v_pred_wolow5.pth').detach().cpu().numpy()
v_pred_wo7 = torch.load('./Figures/AE-G-inversion-results/v_pred_wolow7.pth').detach().cpu().numpy()
vp_tensor = vp_tensor.detach().cpu().numpy()

In [ ]:
# import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Assuming 'ax' is your subplot axis
# Add a rectangle patch on the axis 'ax'
# rect = patches.Rectangle((0, 1.3), width=1, height=0.5, linewidth=1, edgecolor='yellow', facecolor='none')

# # Add the rectangle to the plot
# ax.add_patch(rect)

# # Display the plot
# plt.show()



# AE Stress Test
cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 6))
gs = fig.add_gridspec(2, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred_no1[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred_no2[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(v_pred_no3[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

#------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_no1[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_no2[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(f)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_no3[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.18)

# plt.savefig('./Figures/AE-G-inversion-results-test.png', dpi=300, bbox_inches='tight')
plt.savefig('./Figures/figures/AE-inversion-results-stress-noise.png', dpi=300, bbox_inches='tight')

In [ ]:
# import matplotlib.pyplot as plt
import matplotlib.patches as patches

# AE Stress Test
cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 6))
gs = fig.add_gridspec(2, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred_wo5[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred_wo7[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

#------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_wo5[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_wo7[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.18)

# plt.savefig('./Figures/AE-G-inversion-results-test.png', dpi=300, bbox_inches='tight')
plt.savefig('./Figures/figures/AE-inversion-results-stress-freq.png', dpi=300, bbox_inches='tight')

In [ ]:
# import matplotlib.pyplot as plt
import matplotlib.patches as patches

# AE Stress Test
cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 6))
gs = fig.add_gridspec(2, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred_no1[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred_no2[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(v_pred_no3[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(v_pred_wo5[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
rect = patches.Rectangle((0.02, 1.27), width=1.5, height=0.4, linewidth=2, edgecolor='yellow', facecolor='none')
ax.add_patch(rect)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(v_pred_wo7[0]/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
rect = patches.Rectangle((0.02, 1.27), width=1.5, height=0.4, linewidth=2, edgecolor='yellow', facecolor='none')
ax.add_patch(rect)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

#------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_no1[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(f)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_no2[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(g)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_no3[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(h)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_wo5[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(i)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-v_pred_wo7[0])/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
                vmin=0, vmax=np.abs(vp_tensor[0]-v_pred_no1[0]).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.18)

plt.savefig('./Figures/uncertainty figures/AE-inversion-results-stress.png', dpi=300, bbox_inches='tight')

In [ ]:
train_loss_history_no1 = torch.load('./Figures/train_loss_history/AE_train_loss_history_noised0.1.pth')
train_loss_history_no2 = torch.load('./Figures/train_loss_history/AE_train_loss_history_noised0.2.pth')
train_loss_history_no3 = torch.load('./Figures/train_loss_history/AE_train_loss_history_noised0.3.pth')
train_loss_history_wo5 = torch.load('./Figures/train_loss_history/AE_train_loss_history_wolow5.pth')
train_loss_history_wo7 = torch.load('./Figures/train_loss_history/AE_train_loss_history_wolow7.pth')
train_loss_history = torch.load('./Figures/train_loss_history/AE_train_loss_history.pth')
train_loss_history_no1 = torch.tensor(train_loss_history_no1)
train_loss_history_no2 = torch.tensor(train_loss_history_no2)
train_loss_history_no3 = torch.tensor(train_loss_history_no3)
train_loss_history_wo5 = torch.tensor(train_loss_history_wo5)
train_loss_history_wo7 = torch.tensor(train_loss_history_wo7)
train_loss_history = torch.tensor(train_loss_history)


In [ ]:
z = torch.arange(0, 3000)
fig = plt.figure(figsize=(24, 4))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_no1[0], color='black', linestyle='-', linewidth=1.5, label='$\sigma=0.1\sigma_0$')
ax.plot(z, train_loss_history_no2[0], color='dodgerblue', linestyle='-', linewidth=1.5, label='$\sigma=0.2\sigma_0$')
ax.plot(z, train_loss_history_no3[0], color='orange', linestyle='-', linewidth=1.5, label='$\sigma=0.3\sigma_0$')
ax.plot(z, train_loss_history_wo5[0], color='green', linestyle='-', linewidth=1.5, label='5Hz-filtered')
ax.plot(z, train_loss_history_wo7[0], color='purple', linestyle='-', linewidth=1.5, label='7Hz-filtered')
ax.plot(z, train_loss_history[0], color='red', linestyle='-', linewidth=1.5, label='Raw data')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss all", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=14, ncol=2)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[0, 1])
ax.set_title('(b)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_no1[1], color='black', linestyle='-', linewidth=1.5, label='$\sigma=0.1\sigma_0$')
ax.plot(z, train_loss_history_no2[1], color='dodgerblue', linestyle='-', linewidth=1.5, label='$\sigma=0.2\sigma_0$')
ax.plot(z, train_loss_history_no3[1], color='orange', linestyle='-', linewidth=1.5, label='$\sigma=0.3\sigma_0$')
ax.plot(z, train_loss_history_wo5[1], color='green', linestyle='-', linewidth=1.5, label='5Hz-filtered')
ax.plot(z, train_loss_history_wo7[1], color='purple', linestyle='-', linewidth=1.5, label='7Hz-filtered')
ax.plot(z, train_loss_history[1], color='red', linestyle='-', linewidth=1.5, label='Raw data')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss pixel", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=14, ncol=2)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[0, 2])
ax.set_title('(c)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_no1[2], color='black', linestyle='-', linewidth=1.5, label='$\sigma=0.1\sigma_0$')
ax.plot(z, train_loss_history_no2[2], color='dodgerblue', linestyle='-', linewidth=1.5, label='$\sigma=0.2\sigma_0$')
ax.plot(z, train_loss_history_no3[2], color='orange', linestyle='-', linewidth=1.5, label='$\sigma=0.3\sigma_0$')
ax.plot(z, train_loss_history_wo5[2], color='green', linestyle='-', linewidth=1.5, label='5Hz-filtered')
ax.plot(z, train_loss_history_wo7[2], color='purple', linestyle='-', linewidth=1.5, label='7Hz-filtered')
ax.plot(z, train_loss_history[2], color='red', linestyle='-', linewidth=1.5, label='Raw data')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss perceptual", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=14, ncol=2)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.16,  
                    hspace=0.18)

# plt.savefig('./Figures/uncertainty figures/AE-loss-stress.png', dpi=300, bbox_inches='tight')

In [ ]:
z = torch.arange(0, 3000)

fig = plt.figure(figsize=(22, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, np.sqrt(train_loss_history_no1[3]), color='black', linestyle='-', linewidth=1.5, label='STD0.1')
ax.plot(z, np.sqrt(train_loss_history_no2[3]), color='dodgerblue', linestyle='-', linewidth=1.5, label='STD0.2')
ax.plot(z, np.sqrt(train_loss_history_no3[3]), color='orange', linestyle='-', linewidth=1.5, label='STD0.3')
ax.plot(z, np.sqrt(train_loss_history_wo5[3]), color='green', linestyle='-', linewidth=1.5, label='Wo5hz')
ax.plot(z, np.sqrt(train_loss_history_wo7[3]), color='purple', linestyle='-', linewidth=1.5, label='Wo7hz')
ax.plot(z, np.sqrt(train_loss_history[3]), color='red', linestyle='-', linewidth=1.5, label='train_loss')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Velocity Error [Km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.legend(fontsize=12, ncol=2)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)


# Dropout

In [ ]:
# 汇总展示反演过程的vpred
epochs = [1, 1000, 1500, 2000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/Dropout-G-inversion-results/p0.1/v_pred_train/v_pred_{}.pth'.format(epoch))
# print(v_pred)  

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 15))
gs = fig.add_gridspec(4, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

# ax = fig.add_subplot(gs[0, 2])
# #ax.set_ylabel('Depth (km)', fontsize=16)
# # ax.set_xlabel('Distance [km]', fontsize=16)
# # ax.set_title('(c)', loc='center', fontsize=20)
# im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
#                vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_yticks([])
# ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
# # cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
# ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('Km/s', size=16) 

#==============================================================================================
epochs = [1, 1000, 1500, 2000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/Dropout-G-inversion-results/p0.2/v_pred_train/v_pred_{}.pth'.format(epoch))
    
ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(f)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(g)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

# ax = fig.add_subplot(gs[1, 2])
# # ax.set_ylabel('Depth (km)', fontsize=16)
# # ax.set_xlabel('Distance [km]', fontsize=16)
# # ax.set_title('(h)', loc='center', fontsize=20)
# im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
#                vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_yticks([])
# ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
# # cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 2])
# ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(i)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(j)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 


#==============================================================================================
epochs = [1, 1000, 1500, 2000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/Dropout-G-inversion-results/p0.3/v_pred_train/v_pred_{}.pth'.format(epoch))
    
ax = fig.add_subplot(gs[2, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(k)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(l)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

# ax = fig.add_subplot(gs[2, 2])
# #ax.set_ylabel('Depth (km)', fontsize=16)
# # ax.set_xlabel('Distance [km]', fontsize=16)
# # ax.set_title('(m)', loc='center', fontsize=20)
# im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
#                vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_yticks([])
# ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
# # cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(n)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(o)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

#==============================================================================================
epochs = [1, 1000, 1500, 2000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/Dropout-G-inversion-results/p0.4/v_pred_train/v_pred_{}.pth'.format(epoch))
    
ax = fig.add_subplot(gs[3, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(p)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[3, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(q)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

# ax = fig.add_subplot(gs[3, 2])
# #ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# # ax.set_title('(r)', loc='center', fontsize=20)
# im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
#                vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_yticks([])
# # ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
# # cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[3, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(s)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[3, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(t)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.03,  
                    hspace=0.015)

plt.savefig('./Figures/uncertainty figures/Dropout-inversion-results-train.png', dpi=300, bbox_inches='tight')


In [ ]:
# 汇总展示反演过程的vpred
epochs = [1, 1000, 1500, 2000, 3000]
v_pred = torch.zeros((5, 128, 256))
for s in range(5):
    epoch = epochs[s]
    v_pred[s] = torch.load('./Figures/Dropout-G-inversion-results/p0.1/v_pred_train/v_pred_{}.pth'.format(epoch))
# print(v_pred)  

cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 3))
gs = fig.add_gridspec(1, 5)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred[1].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(v_pred[2].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(v_pred[3].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 4])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(v_pred[4].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.4)

# plt.savefig('./Figures/Dropout-0.1-test-G-inversion-results.png', dpi=300, bbox_inches='tight')


In [ ]:
v_pred0 = torch.load('./Figures/Dropout-G-inversion-results/p0.1v_pred_1.pth')
v_pred1 = torch.load('./Figures/Dropout-G-inversion-results/p0.1v_pred_3000.pth')
v_pred2 = torch.load('./Figures/Dropout-G-inversion-results/p0.2v_pred_3000.pth')
v_pred3 = torch.load('./Figures/Dropout-G-inversion-results/p0.3v_pred_3000.pth')
v_pred4 = torch.load('./Figures/Dropout-G-inversion-results/p0.4v_pred_3000.pth')


cmap = 'RdBu_r'
fig = plt.figure(figsize=(12, 10))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(v_pred0[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(v_pred1[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(v_pred2[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

ax = fig.add_subplot(gs[2, 0])
ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(v_pred3[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 1])
# ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(v_pred4[0].detach().cpu().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.025,  
                    hspace=0.10)

plt.savefig('./Figures/figures/Dropout-inversion-results-simple.png', dpi=300, bbox_inches='tight')


In [ ]:
ps = [0.1, 0.2, 0.3, 0.4]
v_pred = torch.zeros((4, 128, 256))
for s in range(4):
    p = ps[s]
    v_pred[s] = torch.load('./Figures/Dropout-G-inversion-results/p{}/v_pred_train/v_pred_3000.pth'.format(p))

nz = 128
z = dz * torch.arange(0, nz, dtype=torch.float32)/1000                    
v_pred = v_pred/1000
fig = plt.figure(figsize=(22.5, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
nnx0=65
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx0].detach().cpu().numpy(), color='black', linestyle='-', linewidth=1.5, label='p=0.1')
ax.plot(z, v_pred[1, :, nnx0].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='p=0.2')
ax.plot(z, v_pred[2, :, nnx0].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='p=0.3')
ax.plot(z, v_pred[3, :, nnx0].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='p=0.4')
ax.plot(z, vp_tensor[0, :, nnx0].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='true')
ax.set_xlabel("Depth [Km]", fontsize=16)
ax.set_ylabel("Velocity [Km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

ax = fig.add_subplot(gs[0, 1])
nnx1=135
ax.set_title('(b)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx1].detach().cpu().numpy(), color='black', linestyle='-', linewidth=1.5, label='p=0.1')
ax.plot(z, v_pred[1, :, nnx1].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='p=0.2')
ax.plot(z, v_pred[2, :, nnx1].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='p=0.3')
ax.plot(z, v_pred[3, :, nnx1].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='p=0.4')
ax.plot(z, vp_tensor[0, :, nnx0].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='true')
ax.set_xlabel("Depth [Km]", fontsize=16)
ax.set_ylabel("Velocity [Km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

ax = fig.add_subplot(gs[0, 2])
nnx2=240
ax.set_title('(c)', loc='center', fontsize=20)
ax.plot(z, v_pred[0, :, nnx2].detach().cpu().numpy(), color='black', linestyle='-', linewidth=1.5, label='p=0.1')
ax.plot(z, v_pred[1, :, nnx2].detach().cpu().numpy(), color='dodgerblue', linestyle='-', linewidth=1.5, label='p=0.2')
ax.plot(z, v_pred[2, :, nnx2].detach().cpu().numpy(), color='orange', linestyle='-', linewidth=1.5, label='p=0.3')
ax.plot(z, v_pred[3, :, nnx2].detach().cpu().numpy(), color='green', linestyle='-', linewidth=1.5, label='p=0.4')
ax.plot(z, vp_tensor[0, :, nnx2].detach().cpu().numpy()/1000, color='red', linestyle='-', linewidth=1.5, label='true')
ax.set_xlabel("Depth [Km]", fontsize=16)
ax.set_ylabel("Velocity [Km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=1)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)
# ax.minorticks_on()

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.12,  
                    hspace=0.18)

plt.savefig('./Figures/uncertainty figures/Dropout-vprofile.png', dpi=300, bbox_inches='tight')

In [ ]:
train_loss_history_p1 = torch.load('./Figures/train_loss_history/Dropout_0.1_train_loss_history.pth')
train_loss_history_p2 = torch.load('./Figures/train_loss_history/Dropout_0.2_train_loss_history.pth')
train_loss_history_p3 = torch.load('./Figures/train_loss_history/Dropout_0.3_train_loss_history.pth')
train_loss_history_p4 = torch.load('./Figures/train_loss_history/Dropout_0.4_train_loss_history.pth')
train_loss_history = torch.load('./Figures/train_loss_history/AE_train_loss_history.pth')
train_loss_history_p1 = torch.tensor(train_loss_history_p1)
train_loss_history_p2 = torch.tensor(train_loss_history_p2)
train_loss_history_p3 = torch.tensor(train_loss_history_p3)
train_loss_history_p4 = torch.tensor(train_loss_history_p4)
train_loss_history = torch.tensor(train_loss_history)


In [ ]:
z = torch.arange(0, 3000)

fig = plt.figure(figsize=(22.5, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_p1[0], color='black', linestyle='-', linewidth=1.5, label='p=0.1')
ax.plot(z, train_loss_history_p2[0], color='dodgerblue', linestyle='-', linewidth=1.5, label='p=0.2')
ax.plot(z, train_loss_history_p3[0], color='orange', linestyle='-', linewidth=1.5, label='p=0.3')
ax.plot(z, train_loss_history_p4[0], color='green', linestyle='-', linewidth=1.5, label='p=0.4')
ax.plot(z, train_loss_history[0], color='red', linestyle='-', linewidth=1.5, label='AE')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss all", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=3)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[0, 1])
ax.set_title('(b)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_p1[1], color='black', linestyle='-', linewidth=1.5, label='p=0.1')
ax.plot(z, train_loss_history_p2[1], color='dodgerblue', linestyle='-', linewidth=1.5, label='p=0.2')
ax.plot(z, train_loss_history_p3[1], color='orange', linestyle='-', linewidth=1.5, label='p=0.3')
ax.plot(z, train_loss_history_p4[1], color='green', linestyle='-', linewidth=1.5, label='p=0.4')
ax.plot(z, train_loss_history[1], color='red', linestyle='-', linewidth=1.5, label='AE')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss pixel", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=3)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[0, 2])
ax.set_title('(c)', loc='center', fontsize=20)
ax.plot(z, train_loss_history_p1[2], color='black', linestyle='-', linewidth=1.5, label='p=0.1')
ax.plot(z, train_loss_history_p2[2], color='dodgerblue', linestyle='-', linewidth=1.5, label='p=0.2')
ax.plot(z, train_loss_history_p3[2], color='orange', linestyle='-', linewidth=1.5, label='p=0.3')
ax.plot(z, train_loss_history_p4[2], color='green', linestyle='-', linewidth=1.5, label='p=0.4')
ax.plot(z, train_loss_history[2], color='red', linestyle='-', linewidth=1.5, label='AE')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Loss perceptual", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
plt.legend(fontsize=12, ncol=3)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.17,  
                    hspace=0.18)

plt.savefig('./Figures/uncertainty figures/Dropout-loss-p.png.png', dpi=300, bbox_inches='tight')

In [ ]:
z = torch.arange(0, 3000)

fig = plt.figure(figsize=(22, 3.5))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_title('(a)', loc='center', fontsize=20)
ax.plot(z, np.sqrt(train_loss_history_p1[3]), color='black', linestyle='-', linewidth=1.5, label='p0.1')
ax.plot(z, np.sqrt(train_loss_history_p2[3]), color='dodgerblue', linestyle='-', linewidth=1.5, label='p0.2')
ax.plot(z, np.sqrt(train_loss_history_p3[3]), color='orange', linestyle='-', linewidth=1.5, label='p0.3')
ax.plot(z, np.sqrt(train_loss_history_p4[3]), color='green', linestyle='-', linewidth=1.5, label='p0.4')
ax.plot(z, np.sqrt(train_loss_history[3]), color='red', linestyle='-', linewidth=1.5, label='train_loss')
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Velocity Error [Km/s]", fontsize=16)
ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
plt.legend(fontsize=12, ncol=3)
ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)



In [ ]:
Drop_v_samp1 = torch.load('./Figures/v_samp/v_samp_train_Dropout_p0.1.pth').cpu()
Drop_v_samp2 = torch.load('./Figures/v_samp/v_samp_train_Dropout_p0.2.pth').cpu()
Drop_v_samp3 = torch.load('./Figures/v_samp/v_samp_train_Dropout_p0.3.pth').cpu()
Drop_v_samp4 = torch.load('./Figures/v_samp/v_samp_train_Dropout_p0.4.pth').cpu()

# vp_tensor = torch.from_numpy(vp_tensor).cpu()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def add_colorbar(ax, im, bbox_transform, #ax.transAxes
                 width="5%", 
                 height="100%",
                 loc='lower left',
                 bbox_to_anchor=(1.01, 0., 1, 1),
                 borderpad=0,
                 ctitle='',
                orientation=''):
    axins = inset_axes(ax,
                       width=width, 
                       height=height,
                       loc=loc,
                       bbox_to_anchor=bbox_to_anchor,
                       bbox_transform=bbox_transform,
                       borderpad=borderpad)
    cbar = plt.colorbar(im, cax=axins, orientation=orientation)
    axins.set(title=ctitle)
    return cbar

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt

def add_colorbar_new_define(ax, im, bbox_to_anchor=None, width='5%', height='100%', loc='lower center', ctitle=''):
    # 创建一个颜色条的轴，放置在图像下方
    axins = inset_axes(ax,
                       width=width,  # 颜色条的宽度
                       height=height,  # 颜色条的高度
                       loc=loc,  # 颜色条的位置
                       bbox_to_anchor=bbox_to_anchor,  # 调整颜色条的位置
                       bbox_transform=ax.transAxes,  # 相对于父轴定位
                       borderpad=0)  # 边缘填充
    # 添加颜色条
    cbar = plt.colorbar(im, cax=axins, orientation='horizontal')
    if ctitle:
        axins.set_title(ctitle)  # 设置颜色条的标题
    return cbar

# 添加水平颜色条
cbar = add_colorbar_new_define(ax, im, ctitle='km/s')

# 设置颜色条标签的字体大小
cbar.ax.tick_params(labelsize=14)  

# 设置颜色条的标题
cbar.set_label('km/s', size=16)

# 显示图像
plt.show()

In [ ]:
# Dropout 不确定性分析
cmap = 'RdBu_r'
fig = plt.figure(figsize=(30, 11.5))
gs = fig.add_gridspec(3, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(Drop_v_samp1, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(Drop_v_samp2, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(Drop_v_samp3, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(torch.mean(Drop_v_samp4, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 
#-------------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(torch.std(Drop_v_samp1, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(Drop_v_samp1, axis=0).max()/1000)
# ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(f)', loc='center', fontsize=20)
im = ax.imshow(torch.std(Drop_v_samp2, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(Drop_v_samp1, axis=0).max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(g)', loc='center', fontsize=20)
im = ax.imshow(torch.std(Drop_v_samp3, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(Drop_v_samp1, axis=0).max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

ax = fig.add_subplot(gs[1, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(h)', loc='center', fontsize=20)
im = ax.imshow(torch.std(Drop_v_samp4, axis=0).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=torch.std(Drop_v_samp1, axis=0).max()/1000)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

#-------------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[2, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(i)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(Drop_v_samp1, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(Drop_v_samp1, axis=0)).max()/3/1000)
# ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(j)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(Drop_v_samp2, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(Drop_v_samp1, axis=0)).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(k)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(Drop_v_samp3, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(Drop_v_samp1, axis=0)).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[2, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
# ax.set_title('(l)', loc='center', fontsize=20)
im = ax.imshow(np.abs(vp_tensor[0]-torch.mean(Drop_v_samp4, axis=0)).cpu().detach().numpy()/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=0, vmax=(vp_tensor[0]-torch.mean(Drop_v_samp1, axis=0)).max()/3/1000)
ax.set_yticks([])
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.03,  
                    hspace=0.015)

plt.savefig('./Figures/uncertainty figures/Dropout-uncertainty-results.png', dpi=300, bbox_inches='tight')


In [ ]:
import torch
v_samp_AE = torch.load("./Figures/AE-G-inversion-results/v_pred_3000.pth")
v_samp_VAE = torch.load("./Figures/v_samp/v_samp_eval_VAE_N1.pth")
v_samp_Dropout = torch.load("./Figures/v_samp/v_samp_train_Dropout_p0.1.pth")
v_samp_eval_Dropout = torch.load("./Figures/Dropout-G-inversion-results/p0.1v_pred_3000.pth")



In [ ]:
v_samp_Dropout.shape

In [ ]:
fig = plt.figure(figsize=(24, 8))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow((torch.mean(v_samp_Dropout, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r', vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

ax = fig.add_subplot(gs[0, 1])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(((v_samp_eval_Dropout[0]).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r', vmin=vp_tensor.min()/1000, vmax=vp_tensor.max()/1000)
# ax.set_xticks([])
# ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

ax = fig.add_subplot(gs[0, 2])
ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
vmin = (((v_samp_eval_Dropout[0].cpu()-torch.mean(v_samp_Dropout, axis=0).cpu()))/1000).min()  # 最小值
vmax = (((v_samp_eval_Dropout[0].cpu()-torch.mean(v_samp_Dropout, axis=0).cpu()))/1000).max()  # 最大值
im = ax.imshow(np.abs(((v_samp_eval_Dropout[0].cpu()-torch.mean(v_samp_Dropout, axis=0).cpu()).detach().numpy())/1000), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r', vmin=0, vmax=vmax/2)
# ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('km/s', size=16) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.3,  
                    hspace=0.15)

plt.savefig('./Figures/uncertainty figures/Dropout-uncertainty-results-test.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
vi_tensor_true = vi_tensor

In [ ]:
# vi_tensor = vi_tensor_true
vi_tensor = torch.load('./Figures/vi_pred.npz')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_Drop1 = torch.mean(Drop_v_samp1, axis=0)
v_samp_std_Drop1 = torch.std(Drop_v_samp1, axis=0)
v_samp_mean_Drop2 = torch.mean(Drop_v_samp2, axis=0)
v_samp_std_Drop2 = torch.std(Drop_v_samp2, axis=0)
v_samp_mean_Drop3 = torch.mean(Drop_v_samp3, axis=0)
v_samp_std_Drop3 = torch.std(Drop_v_samp3, axis=0)
v_samp_mean_Drop4 = torch.mean(Drop_v_samp4, axis=0)
v_samp_std_Drop4 = torch.std(Drop_v_samp4, axis=0)

nx=150
#模拟创建数据
normal_distribution_Drop1 = torch.distributions.Normal(v_samp_mean_Drop1[:,nx], v_samp_std_Drop1[:,nx])
samples_Drop1 = normal_distribution_Drop1.sample((500,))
normal_distribution_Drop2 = torch.distributions.Normal(v_samp_mean_Drop2[:,nx], v_samp_std_Drop2[:,nx])
samples_Drop2 = normal_distribution_Drop2.sample((500,))
normal_distribution_Drop3 = torch.distributions.Normal(v_samp_mean_Drop3[:,nx], v_samp_std_Drop3[:,nx])
samples_Drop3 = normal_distribution_Drop3.sample((500,))
normal_distribution_Drop4 = torch.distributions.Normal(v_samp_mean_Drop4[:,nx], v_samp_std_Drop4[:,nx])
samples_Drop4 = normal_distribution_Drop4.sample((500,))

data1 = samples_Drop1.T.numpy()
data2 = samples_Drop2.T.numpy()
data3 = samples_Drop3.T.numpy()
data4 = samples_Drop4.T.numpy()

#计算直方图
hist_data1 = []
bins = np.linspace(1500,5500,500)  #设定统一的bins
for i in range(data1.shape[0]):
    hist1, _ = np.histogram(data1[i], bins)  # 计算直方图
    hist_data1.append(hist1)
hist_data1 = np.array(hist_data1)

hist_data2 = []
for i in range(data1.shape[0]):
    hist2, _ = np.histogram(data2[i], bins)  # 计算直方图
    hist_data2.append(hist2)
hist_data2 = np.array(hist_data2)

hist_data3 = []
for i in range(data1.shape[0]):
    hist3, _ = np.histogram(data3[i], bins)  # 计算直方图
    hist_data3.append(hist3)
hist_data3 = np.array(hist_data3)

hist_data4 = []
for i in range(data1.shape[0]):
    hist4, _ = np.histogram(data4[i], bins)  # 计算直方图
    hist_data4.append(hist4)
hist_data4 = np.array(hist_data4)

x = torch.arange(0, 128, dtype=torch.float32)*15/1000

cmap = 'RdBu_r'
fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(1, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=15)
ax.set_xlabel('Velocity [km/s]', fontsize=15)
ax.set_title('(a)', loc='center', fontsize=18)
im = ax.imshow(hist_data1/5, aspect='auto', vmax=1, vmin=0, extent=[1.5, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_Drop1[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='vp_Dropout')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='vp_True')
ax.axhline(y=0.45, color='white', linewidth=1, xmin=0, xmax=1)  #30,60,90,120
ax.axhline(y=0.9, color='white', linewidth=1, xmin=0, xmax=1)  
ax.axhline(y=1.35, color='white', linewidth=1, xmin=0, xmax=1)  
ax.axhline(y=1.8, color='white', linewidth=1, xmin=0, xmax=1)  

# ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.show()

ax = fig.add_subplot(gs[0, 1])
# ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Velocity [km/s]', fontsize=15)
ax.set_title('(b)', loc='center', fontsize=18)
im = ax.imshow(hist_data2/5, aspect='auto', vmax=1, vmin=0, extent=[1.5, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_Drop2[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='vp_Dropout')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='vp_True')
# ax.legend(loc='best')
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.show()

ax = fig.add_subplot(gs[0, 2])
# ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Velocity [km/s]', fontsize=15)
ax.set_title('(c)', loc='center', fontsize=18)
im = ax.imshow(hist_data3/5, aspect='auto', vmax=1, vmin=0, extent=[1.5, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_Drop3[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='vp_Dropout')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='vp_True')
# ax.legend(loc='best')
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.show()

ax = fig.add_subplot(gs[0, 3])
# ax.set_ylabel('Depth [km]', fontsize=16)
ax.set_xlabel('Velocity [km/s]', fontsize=15)
ax.set_title('(d)', loc='center', fontsize=18)
im = ax.imshow(hist_data4/5, aspect='auto', vmax=1, vmin=0, extent=[1.5, 5.5, 1.92, 0]) 
ax.plot(v_samp_mean_Drop4[:,nx].detach().numpy()/1000, x, color='blue', linewidth=1.8, label='Dropout')
ax.plot(vi_tensor_true[0, :,nx].cpu().detach().numpy()/1000, x, color='black', linewidth=1.8, label='Initial')
ax.plot(vp_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='red', linewidth=1.8, label='True')
# ax.plot(vi_tensor[0, :,nx].cpu().detach().numpy()/1000, x, color='orange', label='Initial')

# ax.legend(loc='best')
plt.legend(fontsize=12)
ax.set_yticks([])
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
cbar = add_colorbar(ax, im, ax.transAxes, width="6%")
cbar.ax.tick_params(labelsize=14)  
cbar.set_label('Probability', size=14) 

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.15)

plt.savefig('./Figures/figures/Dropout-uncertainty-vprofile-p.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
v_samp_mean_VAE100 = torch.mean(Drop_v_samp1, axis=0)
v_samp_std_VAE100 = torch.std(Drop_v_samp1, axis=0)
v_samp_mean_VAE1 = torch.mean(Drop_v_samp2, axis=0)
v_samp_std_VAE1 = torch.std(Drop_v_samp2, axis=0)
v_samp_mean_VAE2 = torch.mean(Drop_v_samp3, axis=0)
v_samp_std_VAE2 = torch.std(Drop_v_samp3, axis=0)
v_samp_mean_VAE3 = torch.mean(Drop_v_samp4, axis=0)
v_samp_std_VAE3 = torch.std(Drop_v_samp4, axis=0)

positions = [30, 60, 90, 120]
std_VAE1 = torch.zeros((4, 1))
mean_VAE1 = torch.zeros((4, 1))
std_VAE100 = torch.zeros((4, 1))
mean_VAE100 = torch.zeros((4, 1))
std_VAE2 = torch.zeros((4, 1))
mean_VAE2 = torch.zeros((4, 1))
std_VAE3 = torch.zeros((4, 1))
mean_VAE3 = torch.zeros((4, 1))
for s in range(4):
    po = positions[s]
    std_VAE1[s] = v_samp_std_VAE1[po,150]/1000
    mean_VAE1[s] = v_samp_mean_VAE1[po,150]/1000
    std_VAE100[s] = v_samp_std_VAE100[po,150]/1000
    mean_VAE100[s] = v_samp_mean_VAE100[po,150]/1000
    std_VAE2[s] = v_samp_std_VAE2[po,150]/1000
    mean_VAE2[s] = v_samp_mean_VAE2[po,150]/1000
    std_VAE3[s] = v_samp_std_VAE3[po,150]/1000
    mean_VAE3[s] = v_samp_mean_VAE3[po,150]/1000

In [ ]:
from scipy.stats import norm

z = np.linspace(1500, 5500, 3000)/1000

fig = plt.figure(figsize=(4.5, 11))
gs = fig.add_gridspec(4, 1)
ax = fig.add_subplot(gs[0, 0])
pdf1 = norm.pdf(z, mean_VAE1[0].cpu().detach().numpy(), std_VAE1[0].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[0].cpu().detach().numpy(), std_VAE2[0].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[0].cpu().detach().numpy(), std_VAE3[0].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[0].cpu().detach().numpy(), std_VAE100[0].cpu().detach().numpy())

ax.set_title('(i)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='dodgerblue')
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='red')
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple')
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green')
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# ax.set_xlabel("Velocity [Km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
plt.legend(fontsize=12, ncol=1, loc='upper right')
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)


ax = fig.add_subplot(gs[1, 0])
pdf1 = norm.pdf(z, mean_VAE1[1].cpu().detach().numpy(), std_VAE1[1].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[1].cpu().detach().numpy(), std_VAE2[1].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[1].cpu().detach().numpy(), std_VAE3[1].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[1].cpu().detach().numpy(), std_VAE100[1].cpu().detach().numpy())

ax.set_title('(ii)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='dodgerblue')
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='red')
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple')
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green')
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# ax.set_xlabel("Velocity [Km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
# plt.legend(fontsize=12, ncol=1, loc='upper right')
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[2, 0])
pdf1 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE1[2].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[2].cpu().detach().numpy(), std_VAE2[2].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE3[2].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[2].cpu().detach().numpy(), std_VAE100[2].cpu().detach().numpy())

ax.set_title('(iii)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='dodgerblue')
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='red')
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple')
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green')
# ax.set_xlabel("Velocity [Km/s]", fontsize=16)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.legend(fontsize=12, ncol=1)
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

ax = fig.add_subplot(gs[3, 0])
pdf1 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE1[2].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[2].cpu().detach().numpy(), std_VAE2[2].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE3[2].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[2].cpu().detach().numpy(), std_VAE100[2].cpu().detach().numpy())

ax.set_title('(iv)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='dodgerblue')
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='red')
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple')
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green')
ax.set_xlabel("Velocity [km/s]", fontsize=18)
ax.set_ylabel("PDF", fontsize=16)
ax.tick_params(axis='x', labelsize=14) 
ax.tick_params(axis='y', labelsize=14) 
# plt.legend(fontsize=12, ncol=1)

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.14,  
                    hspace=0.3)



plt.savefig('./Figures/uncertainty figures/Dropout-uncertainty-point-dis-peitu.png', dpi=300, bbox_inches='tight')


In [ ]:
from scipy.stats import norm

z = np.linspace(1500, 5500, 3000)/1000
zz = 30
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.04
fig = plt.figure(figsize=(20, 2.5))
gs = fig.add_gridspec(1, 4)
ax = fig.add_subplot(gs[0, 0])
pdf1 = norm.pdf(z, mean_VAE1[0].cpu().detach().numpy(), std_VAE1[0].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[0].cpu().detach().numpy(), std_VAE2[0].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[0].cpu().detach().numpy(), std_VAE3[0].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[0].cpu().detach().numpy(), std_VAE100[0].cpu().detach().numpy())

ax.set_title('(i)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='orange', zorder=1)
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='dodgerblue', zorder=1)
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple', zorder=1)
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green', zorder=1)
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True', zorder=2)
ax.tick_params(axis='x', labelsize=15) 
ax.tick_params(axis='y', labelsize=15) 
ax.set_xlabel("Velocity [km/s]", fontsize=18)
ax.set_ylabel("PDF", fontsize=18)
plt.legend(fontsize=15, ncol=1, loc='upper right')
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)

zz = 60
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.03
ax = fig.add_subplot(gs[0, 1])
pdf1 = norm.pdf(z, mean_VAE1[1].cpu().detach().numpy(), std_VAE1[1].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[1].cpu().detach().numpy(), std_VAE2[1].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[1].cpu().detach().numpy(), std_VAE3[1].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[1].cpu().detach().numpy(), std_VAE100[1].cpu().detach().numpy())

ax.set_title('(ii)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='orange', zorder=1)
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='dodgerblue', zorder=1)
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple', zorder=1)
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green', zorder=1)
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True', zorder=2)
ax.tick_params(axis='x', labelsize=15) 
ax.tick_params(axis='y', labelsize=15) 
ax.set_xlabel("Velocity [km/s]", fontsize=18)
ax.set_ylabel("PDF", fontsize=18)
# plt.legend(fontsize=12, ncol=1, loc='upper right')
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)


zz = 90
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.02
ax = fig.add_subplot(gs[0, 2])
pdf1 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE1[2].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[2].cpu().detach().numpy(), std_VAE2[2].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE3[2].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[2].cpu().detach().numpy(), std_VAE100[2].cpu().detach().numpy())

ax.set_title('(iii)', loc='center', fontsize=20)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='orange', zorder=1)
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='dodgerblue', zorder=1)
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple', zorder=1)
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green', zorder=1)
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True', zorder=2)
ax.set_xlabel("Velocity [km/s]", fontsize=18)
ax.set_ylabel("PDF", fontsize=18)
ax.tick_params(axis='x', labelsize=15) 
ax.tick_params(axis='y', labelsize=15) 
# plt.legend(fontsize=12, ncol=1)
# ax.grid(True, which='both', linestyle='--', color='grey', linewidth=.8, alpha=1.0)


zz = 120
xx = 150
x_vp_scat = vp_tensor[:, zz, xx].cpu().detach().numpy() 
y_scat = np.ones(1)*0.02
ax = fig.add_subplot(gs[0, 3])
pdf1 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE1[2].cpu().detach().numpy())
pdf2 = norm.pdf(z, mean_VAE2[2].cpu().detach().numpy(), std_VAE2[2].cpu().detach().numpy())
pdf3 = norm.pdf(z, mean_VAE1[2].cpu().detach().numpy(), std_VAE3[2].cpu().detach().numpy())
pdf4 = norm.pdf(z, mean_VAE100[2].cpu().detach().numpy(), std_VAE100[2].cpu().detach().numpy())

ax.set_title('(iv)', loc='center', fontsize=20)
plt.scatter(x_vp_scat/1000, y_scat, s=158, alpha=1, marker="*", color='red', label='True', zorder=2)
ax.plot(z, pdf4, label='p=0.1', linewidth=2, color='orange', zorder=1)
ax.plot(z, pdf1, label='p=0.2', linewidth=2, color='dodgerblue', zorder=1)
ax.plot(z, pdf2, label='p=0.3', linewidth=2, color='purple', zorder=1)
ax.plot(z, pdf3, label='p=0.4', linewidth=2, color='green', zorder=1)
ax.set_xlabel("Velocity [km/s]", fontsize=18)
ax.set_ylabel("PDF", fontsize=18)
ax.tick_params(axis='x', labelsize=15) 
ax.tick_params(axis='y', labelsize=15) 
# plt.legend(fontsize=12, ncol=1)

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.18,  
                    hspace=0.3)


plt.savefig('./Figures/figures/Dropout-uncertainty-point-dis-peitu.png', dpi=300, bbox_inches='tight')


In [ ]:
import scipy
skew_v_Drop1 = scipy.stats.skew(Drop_v_samp1)  #偏度
skew_v_Drop2 = scipy.stats.skew(Drop_v_samp2)
skew_v_Drop3 = scipy.stats.skew(Drop_v_samp3)
skew_v_Drop4 = scipy.stats.skew(Drop_v_samp4)

kurtosis_v_Drop1 = scipy.stats.kurtosis(Drop_v_samp1)  #峰度
kurtosis_v_Drop2 = scipy.stats.kurtosis(Drop_v_samp2)
kurtosis_v_Drop3 = scipy.stats.kurtosis(Drop_v_samp3)
kurtosis_v_Drop4 = scipy.stats.kurtosis(Drop_v_samp4)



In [ ]:
'''
度 (Skewness)
偏度测量的是数据分布的不对称程度：
偏度约等于0: 数据分布接近对称，意味着数据两端尾部的长度大致相同。
偏度大于0: 数据分布右偏（正偏态），表明数据的右尾（较大值）较长或较重。图形上看，分布的右侧尾部会比左侧尾部更拉长。
偏度小于0: 数据分布左偏（负偏态），表明数据的左尾（较小值）较长或较重。图形上看，分布的左侧尾部会比右侧尾部更拉长。
抽象点来说，偏度描述的是数据分布的平均值（Mean）和中位数（Median）的偏离程度，以及分布的对称性。
峰度 (Kurtosis)
峰度衡量的是数据分布尾部的厚度和尖锐程度：
峰度约等于0 (或在标准化后等于3): 当用“绝对峰度”表示时，数据分布呈现出正态分布的特性，正态分布的峰度是3。当用“超额峰度”表示（减去3的标准化峰度）时，数据分布的峰度是0，表示与正态分布的尾部厚度相近。
峰度大于0 (或在标准化后大于3): 数据分布是尖顶峰态（Leptokurtic），意味着数据分布的尾部比正态分布的尾部厚重，峰值较尖锐，中间的数据较集中。
峰度小于0 (或在标准化后小于3): 数据分布是平顶峰态（Platykurtic），意味着数据分布的尾部比正态分布的尾部薄，峰值较平缓，中间的数据较分散。
峰度主要描述的是异常值（Outliers）的出现概率和分布的尾部特性，高峰度值可能意味着异常值更多。
'''

In [ ]:
# Dropout 不确定性分析
cmap = 'RdBu_r'
fig = plt.figure(figsize=(24, 6))
gs = fig.add_gridspec(2, 4)
ax = fig.add_subplot(gs[0, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(a)', loc='center', fontsize=20)
im = ax.imshow(skew_v_Drop1, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=-skew_v_Drop1.max()/3, vmax=skew_v_Drop1.max()/3)
ax.set_xticks([])
# ax.tick_params(axis='x', labelsize=12) 
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(b)', loc='center', fontsize=20)
im = ax.imshow(skew_v_Drop2, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=-skew_v_Drop1.max()/3, vmax=skew_v_Drop1.max()/3)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(c)', loc='center', fontsize=20)
im = ax.imshow(skew_v_Drop3, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
              vmin=-skew_v_Drop1.max()/3, vmax=skew_v_Drop1.max()/3)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[0, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(d)', loc='center', fontsize=20)
im = ax.imshow(skew_v_Drop4, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=-skew_v_Drop1.max()/3, vmax=skew_v_Drop1.max()/3)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 
#-------------------------------------------------------------------------------------------------------------------------------

ax = fig.add_subplot(gs[1, 0])
ax.set_ylabel('Depth [km]', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(e)', loc='center', fontsize=20)
im = ax.imshow(kurtosis_v_Drop1, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=-kurtosis_v_Drop1.max()/4, vmax=kurtosis_v_Drop1.max()/4)
# ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='y', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='$Km/s$')

ax = fig.add_subplot(gs[1, 1])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(f)', loc='center', fontsize=20)
im = ax.imshow(kurtosis_v_Drop2, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=-kurtosis_v_Drop1.max()/4, vmax=kurtosis_v_Drop1.max()/4)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

ax = fig.add_subplot(gs[1, 2])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(g)', loc='center', fontsize=20)
im = ax.imshow(kurtosis_v_Drop3, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
              vmin=-kurtosis_v_Drop1.max()/4, vmax=kurtosis_v_Drop1.max()/4)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 

ax = fig.add_subplot(gs[1, 3])
#ax.set_ylabel('Depth (km)', fontsize=16)
# ax.set_xlabel('Distance [km]', fontsize=16)
ax.set_title('(h)', loc='center', fontsize=20)
im = ax.imshow(kurtosis_v_Drop4, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap=cmap,
               vmin=-kurtosis_v_Drop1.max()/4, vmax=kurtosis_v_Drop1.max()/4)
ax.set_yticks([])
ax.set_xticks([])
ax.tick_params(axis='x', labelsize=12) 
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.tick_params(labelsize=12)  
cbar.set_label('Km/s', size=14) 


plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05,  
                    hspace=0.2)
plt.show()
# plt.savefig('./Figures/Dropout-0.1-test-G-inversion-results.png', dpi=300, bbox_inches='tight')


In [ ]:
import torchvision
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = torch.load("./shots_marmousi_I_2048.npz").to(device).type(dtype=torch.float32)  #[1,12,1024,256]


In [ ]:
model = MyModel().to(device)
optimizer = torch.optim.AdamW([{'params': model.parameters()}], 
                               lr=0.00001, betas=(0.9, 0.999), weight_decay=0.0001)
_, model, _, _ = load_state(model, optimizer, "./checkpoints-VAE-marmousi-G/checkpoint-VAE-marmousi-1-3000.pth") 
model = model.to(device)
model_vgg = torchvision.models.vgg16(pretrained=True).features[:13].to(device) #第13层输出--conv5的结果。


In [ ]:
samples = 5
v_samp = torch.zeros((samples, 128, 256))
for s in range(samples):
    v_pred, _, _ = test(model, data, device)
    v_samp[s] = v_pred


In [ ]:
fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean epoch 3000', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
# 输出并保存每个epoch隐码的均值和标准差
v_pred, Z_mean, log_Z_std = test(model, data, device)
Z_std = torch.exp(0.5 * log_Z_std)


In [ ]:
torch.save(Z_mean, './Z_mean_VAE/Z_mean_epoch_{}'.format(3000))
torch.save(Z_std, './Z_std_VAE/Z_std_epoch_{}'.format(3000))


In [ ]:
from scipy.stats import norm

# 定义均值和方差
n=255
Z_mean_VAE_one = Z_mean[0,n]
Z_std_VAE_one = Z_std[0,n]

# 生成 x 值  
z_VAE = np.linspace(-10, 10, 3000)
z = np.linspace(-10, 10, 3000)

# 计算正态分布的概率密度函数值
pdf_VAE_Z = norm.pdf(z_VAE, Z_mean_VAE_one.cpu().detach().numpy(), Z_std_VAE_one.cpu().detach().numpy())
pdf_Z = norm.pdf(z, 0, 1)

plt.plot(z_VAE, pdf_VAE_Z, label='Z_VAE Distribution', color='darkblue')
plt.plot(z, pdf_Z, label='normal Distribution', color='red')


plt.legend()
plt.title('Normal Distribution epoch 3000')
plt.xlabel('V [m/s]')
plt.ylabel('Density')
plt.show()

## "./checkpoints-VAE-marmousi-GN(0,100)/checkpoint-VAE-marmousi-1-3000.pth"

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
# plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy()/1000000000000000, label="KLD")
plt.legend()

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
# plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
# plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy(), label="KLD")
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(1,5000), train_loss_history[1:5000, 0].numpy(), label="all")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 1].numpy(), label="recon_pixel")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 2].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[4, 0:n].numpy(), label="recon_vp")

plt.legend()


In [ ]:
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred")
im = ax.imshow((v_pred[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
# ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_init")
# im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp")
im = ax.imshow(((vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[1, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vi")
im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[2, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vp")
im = ax.imshow(((v_pred[0]-vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vp_tensor[0])/1000).max()  # 最小值
vmax = ((v_pred[0]-vp_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)


ax = fig.add_subplot(gs[2, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vi")
im = ax.imshow(((v_pred[0]-vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vi_tensor[0])/1000).max()   # 最小值
vmax = ((v_pred[0]-vi_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)



In [ ]:
vvmin = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).min()  # 最小值
vvmax = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).max()  # 最大值
print(vvmin, vvmax)

fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()/4  # 最大值
# vmax = 0.2  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE = torch.mean(v_samp, axis=0)
v_samp_std_VAE = torch.std(v_samp, axis=0)

nx=205
#模拟创建数据
normal_distribution_VAE = torch.distributions.Normal(v_samp_mean_VAE[:,nx], v_samp_std_VAE[:,nx])
samples_VAE = normal_distribution_VAE.sample((300,))
# data = np.random.randn(5,1000)  # 5 组数据，每组1000个数据
data = samples_VAE.T.numpy()

#计算直方图
hist_data = []
bins = np.linspace(1500,5500,300)  #设定统一的bins
for i in range(data.shape[0]):
    hist, _ = np.histogram(data[i], bins)  # 计算直方图
    hist_data.append(hist)
hist_data = np.array(hist_data)

# 使用imshow()绘制
plt.figure(figsize=(5, 10))
plt.imshow(hist_data/3, aspect='auto', vmax=10, vmin=0, extent=[bins.min(), bins.max(), 1.92, 0]) 
plt.colorbar(shrink=1, fraction=0.1, aspect=50, label='Probability [%]')
plt.xlabel('V [Km/s]')
plt.ylabel('Z [Km]')
plt.title('Histogram heatmap')

x = torch.arange(0, 128, dtype=torch.float32)*15/1000
plt.plot(v_samp_mean_VAE[:,nx].detach().numpy(), x, color='blue', label='vp_VAE')
plt.plot(v_samp_mean_Dropout[:,nx].detach().numpy(), x, color='orange', label='vp_Dropout')
plt.plot(vp_tensor[0, :,nx].cpu().detach().numpy(), x, color='red', label='vp_True')
plt.legend()
plt.show()


## "./checkpoints-VAE-marmousi-GN(0,1)/checkpoint-VAE-marmousi-1-3000.pth"

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy()/1000000000000000, label="KLD")
plt.legend()

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
# plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
# plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy(), label="KLD")
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(1,5000), train_loss_history[1:5000, 0].numpy(), label="all")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 1].numpy(), label="recon_pixel")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 2].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[4, 0:n].numpy(), label="recon_vp")

plt.legend()


In [ ]:
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred")
im = ax.imshow((v_pred[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
# ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_init")
# im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp")
im = ax.imshow(((vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[1, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vi")
im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[2, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vp")
im = ax.imshow(((v_pred[0]-vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vp_tensor[0])/1000).max()  # 最小值
vmax = ((v_pred[0]-vp_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)


ax = fig.add_subplot(gs[2, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vi")
im = ax.imshow(((v_pred[0]-vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vi_tensor[0])/1000).max()   # 最小值
vmax = ((v_pred[0]-vi_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)



In [ ]:
vvmin = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).min()  # 最小值
vvmax = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).max()  # 最大值
print(vvmin, vvmax)

fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()/4  # 最大值
# vmax = 0.2  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE = torch.mean(v_samp, axis=0)
v_samp_std_VAE = torch.std(v_samp, axis=0)

nx=205
#模拟创建数据
normal_distribution_VAE = torch.distributions.Normal(v_samp_mean_VAE[:,nx], v_samp_std_VAE[:,nx])
samples_VAE = normal_distribution_VAE.sample((300,))
# data = np.random.randn(5,1000)  # 5 组数据，每组1000个数据
data = samples_VAE.T.numpy()

#计算直方图
hist_data = []
bins = np.linspace(1500,5500,300)  #设定统一的bins
for i in range(data.shape[0]):
    hist, _ = np.histogram(data[i], bins)  # 计算直方图
    hist_data.append(hist)
hist_data = np.array(hist_data)

# 使用imshow()绘制
plt.figure(figsize=(5, 10))
plt.imshow(hist_data/3, aspect='auto', vmax=10, vmin=0, extent=[bins.min(), bins.max(), 1.92, 0]) 
plt.colorbar(shrink=1, fraction=0.1, aspect=50, label='Probability [%]')
plt.xlabel('V [Km/s]')
plt.ylabel('Z [Km]')
plt.title('Histogram heatmap')

x = torch.arange(0, 128, dtype=torch.float32)*15/1000
plt.plot(v_samp_mean_VAE[:,nx].detach().numpy(), x, color='blue', label='vp_VAE')
plt.plot(v_samp_mean_Dropout[:,nx].detach().numpy(), x, color='orange', label='vp_Dropout')
plt.plot(vp_tensor[0, :,nx].cpu().detach().numpy(), x, color='red', label='vp_True')
plt.legend()
plt.show()


## "./checkpoints-VAE-marmousi-GN(0,0.5)/checkpoint-VAE-marmousi-1-3000.pth"

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
# plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy()/1000000000000000, label="KLD")
plt.legend()

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
# plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
# plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy(), label="KLD")
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(1,5000), train_loss_history[1:5000, 0].numpy(), label="all")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 1].numpy(), label="recon_pixel")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 2].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[4, 0:n].numpy(), label="recon_vp")

plt.legend()


In [ ]:
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred")
im = ax.imshow((v_pred[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
# ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_init")
# im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp")
im = ax.imshow(((vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[1, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vi")
im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[2, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vp")
im = ax.imshow(((v_pred[0]-vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vp_tensor[0])/1000).max()  # 最小值
vmax = ((v_pred[0]-vp_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)


ax = fig.add_subplot(gs[2, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vi")
im = ax.imshow(((v_pred[0]-vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vi_tensor[0])/1000).max()   # 最小值
vmax = ((v_pred[0]-vi_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)



In [ ]:
vvmin = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).min()  # 最小值
vvmax = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).max()  # 最大值
print(vvmin, vvmax)

fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()/4  # 最大值
# vmax = 0.2  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE = torch.mean(v_samp, axis=0)
v_samp_std_VAE = torch.std(v_samp, axis=0)

nx=205
#模拟创建数据
normal_distribution_VAE = torch.distributions.Normal(v_samp_mean_VAE[:,nx], v_samp_std_VAE[:,nx])
samples_VAE = normal_distribution_VAE.sample((300,))
# data = np.random.randn(5,1000)  # 5 组数据，每组1000个数据
data = samples_VAE.T.numpy()

#计算直方图
hist_data = []
bins = np.linspace(1500,5500,300)  #设定统一的bins
for i in range(data.shape[0]):
    hist, _ = np.histogram(data[i], bins)  # 计算直方图
    hist_data.append(hist)
hist_data = np.array(hist_data)

# 使用imshow()绘制
plt.figure(figsize=(5, 10))
plt.imshow(hist_data/3, aspect='auto', vmax=10, vmin=0, extent=[bins.min(), bins.max(), 1.92, 0]) 
plt.colorbar(shrink=1, fraction=0.1, aspect=50, label='Probability [%]')
plt.xlabel('V [Km/s]')
plt.ylabel('Z [Km]')
plt.title('Histogram heatmap')

x = torch.arange(0, 128, dtype=torch.float32)*15/1000
plt.plot(v_samp_mean_VAE[:,nx].detach().numpy(), x, color='blue', label='vp_VAE')
plt.plot(v_samp_mean_Dropout[:,nx].detach().numpy(), x, color='orange', label='vp_Dropout')
plt.plot(vp_tensor[0, :,nx].cpu().detach().numpy(), x, color='red', label='vp_True')
plt.legend()
plt.show()


## "./checkpoints-VAE-marmousi-GN(0,0.1)/checkpoint-VAE-marmousi-1-3000.pth"

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
# plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy()/1000000000000000, label="KLD")
plt.legend()

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
# plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
# plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy(), label="KLD")
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(1,5000), train_loss_history[1:5000, 0].numpy(), label="all")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 1].numpy(), label="recon_pixel")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 2].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[4, 0:n].numpy(), label="recon_vp")

plt.legend()


In [ ]:
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred")
im = ax.imshow((v_pred[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
# ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_init")
# im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp")
im = ax.imshow(((vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[1, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vi")
im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[2, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vp")
im = ax.imshow(((v_pred[0]-vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vp_tensor[0])/1000).max()  # 最小值
vmax = ((v_pred[0]-vp_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)


ax = fig.add_subplot(gs[2, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vi")
im = ax.imshow(((v_pred[0]-vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vi_tensor[0])/1000).max()   # 最小值
vmax = ((v_pred[0]-vi_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)



In [ ]:
vvmin = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).min()  # 最小值
vvmax = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).max()  # 最大值
print(vvmin, vvmax)

fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()/4  # 最大值
# vmax = 0.2  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE = torch.mean(v_samp, axis=0)
v_samp_std_VAE = torch.std(v_samp, axis=0)

nx=205
#模拟创建数据
normal_distribution_VAE = torch.distributions.Normal(v_samp_mean_VAE[:,nx], v_samp_std_VAE[:,nx])
samples_VAE = normal_distribution_VAE.sample((300,))
# data = np.random.randn(5,1000)  # 5 组数据，每组1000个数据
data = samples_VAE.T.numpy()

#计算直方图
hist_data = []
bins = np.linspace(1500,5500,300)  #设定统一的bins
for i in range(data.shape[0]):
    hist, _ = np.histogram(data[i], bins)  # 计算直方图
    hist_data.append(hist)
hist_data = np.array(hist_data)

# 使用imshow()绘制
plt.figure(figsize=(5, 10))
plt.imshow(hist_data/3, aspect='auto', vmax=10, vmin=0, extent=[bins.min(), bins.max(), 1.92, 0]) 
plt.colorbar(shrink=1, fraction=0.1, aspect=50, label='Probability [%]')
plt.xlabel('V [Km/s]')
plt.ylabel('Z [Km]')
plt.title('Histogram heatmap')

x = torch.arange(0, 128, dtype=torch.float32)*15/1000
plt.plot(v_samp_mean_VAE[:,nx].detach().numpy(), x, color='blue', label='vp_VAE')
plt.plot(v_samp_mean_Dropout[:,nx].detach().numpy(), x, color='orange', label='vp_Dropout')
plt.plot(vp_tensor[0, :,nx].cpu().detach().numpy(), x, color='red', label='vp_True')
plt.legend()
plt.show()


## "./checkpoints-VAE-marmousi-G-N(0,0.01)/checkpoint-VAE-marmousi-1-3000.pth"

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
# plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy()/1000000000000000, label="KLD")
plt.legend()

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# # plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
# plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
# plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[3, 0:n].numpy(), label="KLD")
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(1,5000), train_loss_history[1:5000, 0].numpy(), label="all")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 1].numpy(), label="recon_pixel")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 2].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[4, 0:n].numpy(), label="recon_vp")

plt.legend()


In [ ]:
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred")
im = ax.imshow((v_pred[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
# ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_init")
# im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp")
im = ax.imshow(((vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[1, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vi")
im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[2, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vp")
im = ax.imshow(((v_pred[0]-vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vp_tensor[0])/1000).max()  # 最小值
vmax = ((v_pred[0]-vp_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)


ax = fig.add_subplot(gs[2, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vi")
im = ax.imshow(((v_pred[0]-vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vi_tensor[0])/1000).max()   # 最小值
vmax = ((v_pred[0]-vi_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)



In [ ]:
vvmin = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).min()  # 最小值
vvmax = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).max()  # 最大值
print(vvmin, vvmax)

fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()/4  # 最大值
# vmax = 0.2  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE = torch.mean(v_samp, axis=0)
v_samp_std_VAE = torch.std(v_samp, axis=0)

nx=205
#模拟创建数据
normal_distribution_VAE = torch.distributions.Normal(v_samp_mean_VAE[:,nx], v_samp_std_VAE[:,nx])
samples_VAE = normal_distribution_VAE.sample((300,))
# data = np.random.randn(5,1000)  # 5 组数据，每组1000个数据
data = samples_VAE.T.numpy()

#计算直方图
hist_data = []
bins = np.linspace(1500,5500,300)  #设定统一的bins
for i in range(data.shape[0]):
    hist, _ = np.histogram(data[i], bins)  # 计算直方图
    hist_data.append(hist)
hist_data = np.array(hist_data)

# 使用imshow()绘制
plt.figure(figsize=(5, 10))
plt.imshow(hist_data/3, aspect='auto', vmax=10, vmin=0, extent=[bins.min(), bins.max(), 1.92, 0]) 
plt.colorbar(shrink=1, fraction=0.1, aspect=50, label='Probability [%]')
plt.xlabel('V [Km/s]')
plt.ylabel('Z [Km]')
plt.title('Histogram heatmap')

x = torch.arange(0, 128, dtype=torch.float32)*15/1000
plt.plot(v_samp_mean_VAE[:,nx].detach().numpy(), x, color='blue', label='vp_VAE')
plt.plot(v_samp_mean_Dropout[:,nx].detach().numpy(), x, color='orange', label='vp_Dropout')
plt.plot(vp_tensor[0, :,nx].cpu().detach().numpy(), x, color='red', label='vp_True')
plt.legend()
plt.show()


## "./checkpoints-VAE-marmousi-G-N(0,0.001)/checkpoint-VAE-marmousi-1-3000.pth"

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
# plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy()/1000000000000000, label="KLD")
plt.legend()

In [ ]:
n=3000
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(0,n), train_loss_history[0, 0:n].numpy()/10000000000000000000, label="all")
# plt.plot(np.arange(0,n), train_loss_history[1, 0:n].numpy()*1000, label="recon_pixel")
# plt.plot(np.arange(0,n), train_loss_history[2, 0:n].numpy(), label="recon_percep")
plt.plot(np.arange(50,n), train_loss_history[3, 50:n].numpy(), label="KLD")
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
# plt.plot(np.arange(1,5000), train_loss_history[1:5000, 0].numpy(), label="all")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 1].numpy(), label="recon_pixel")
# plt.plot(np.arange(1,1000), train_loss_history[1:1000, 2].numpy(), label="recon_percep")
plt.plot(np.arange(0,n), train_loss_history[4, 0:n].numpy(), label="recon_vp")

plt.legend()


In [ ]:
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(3, 2)
ax = fig.add_subplot(gs[0, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred")
im = ax.imshow((v_pred[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
# ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_init")
# im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
# cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp")
im = ax.imshow(((vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[1, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vi")
im = ax.imshow(((vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)

ax = fig.add_subplot(gs[2, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vp")
im = ax.imshow(((v_pred[0]-vp_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vp_tensor[0])/1000).max()  # 最小值
vmax = ((v_pred[0]-vp_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)


ax = fig.add_subplot(gs[2, 0])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred-vi")
im = ax.imshow(((v_pred[0]-vi_tensor[0]).cpu().detach().numpy() )/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=1, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%", ctitle='km/s')

# 设置colorbar的取值范围
vmin = -((v_pred[0]-vi_tensor[0])/1000).max()   # 最小值
vmax = ((v_pred[0]-vi_tensor[0])/1000).max()   # 最大值
im.set_clim(vmin, vmax)



In [ ]:
vvmin = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).min()  # 最小值
vvmax = (((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()))/1000).max()  # 最大值
print(vvmin, vvmax)

fig = plt.figure(figsize=(21, 9.5))
gs = fig.add_gridspec(2, 2)
fontsize=16
ax = fig.add_subplot(gs[0, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_mean")
im = ax.imshow((torch.mean(v_samp, axis=0).cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[0, 1])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="v_pred_std")
im = ax.imshow((torch.std(v_samp/1000, axis=0).cpu().detach().numpy()), extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((torch.std(v_samp/1000, axis=0))).min()  # 最小值
vmax = ((torch.std(v_samp/1000, axis=0))).max()/4  # 最大值
# vmax = 0.2  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('v_pred_std', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 0])
# ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp_tensor")
im = ax.imshow((vp_tensor[0].cpu().detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# 设置colorbar的取值范围
vmin = ((vp_tensor[0])/1000).min()  # 最小值
vmax = ((vp_tensor[0])/1000).max()  # 最大值
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)


ax = fig.add_subplot(gs[1, 1])
ax.set(ylabel="Depth z[km]", xlabel="Distance x[km]", title="vp-v_pred_mean")
im = ax.imshow(((vp_tensor[0].cpu()-torch.mean(v_samp, axis=0).cpu()).detach().numpy())/1000, extent=[0, nx*dz/1000, nz*dz/1000, 0], aspect=0.8, cmap='RdBu_r')
cbar = add_colorbar(ax, im, ax.transAxes, width="3%")
cbar.ax.set_title('km/s', size=fontsize)
cbar.ax.tick_params(labelsize=fontsize)
# cbar.set_label('km/s', size=fontsize)
# 设置colorbar的取值范围
vmin = -vvmax
vmax = vvmax
im.set_clim(vmin, vmax)
ax.set_xlabel("Distance x[km]", fontsize=fontsize)
ax.set_title('vp-v_pred_mean', fontsize=fontsize)
ax.set_ylabel("Depth z[km]",fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v_samp_mean_VAE = torch.mean(v_samp, axis=0)
v_samp_std_VAE = torch.std(v_samp, axis=0)

nx=205
#模拟创建数据
normal_distribution_VAE = torch.distributions.Normal(v_samp_mean_VAE[:,nx], v_samp_std_VAE[:,nx])
samples_VAE = normal_distribution_VAE.sample((300,))
# data = np.random.randn(5,1000)  # 5 组数据，每组1000个数据
data = samples_VAE.T.numpy()

#计算直方图
hist_data = []
bins = np.linspace(1500,5500,300)  #设定统一的bins
for i in range(data.shape[0]):
    hist, _ = np.histogram(data[i], bins)  # 计算直方图
    hist_data.append(hist)
hist_data = np.array(hist_data)

# 使用imshow()绘制
plt.figure(figsize=(5, 10))
plt.imshow(hist_data/3, aspect='auto', vmax=10, vmin=0, extent=[bins.min(), bins.max(), 1.92, 0]) 
plt.colorbar(shrink=1, fraction=0.1, aspect=50, label='Probability [%]')
plt.xlabel('V [Km/s]')
plt.ylabel('Z [Km]')
plt.title('Histogram heatmap')

x = torch.arange(0, 128, dtype=torch.float32)*15/1000
plt.plot(v_samp_mean_VAE[:,nx].detach().numpy(), x, color='blue', label='vp_VAE')
plt.plot(v_samp_mean_Dropout[:,nx].detach().numpy(), x, color='orange', label='vp_Dropout')
plt.plot(vp_tensor[0, :,nx].cpu().detach().numpy(), x, color='red', label='vp_True')
plt.legend()
plt.show()


In [ ]:
from scipy.stats import norm
import torchvision


In [ ]:
# 训练模型
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
data = torch.load("./shots_marmousi_I_2048.npz").to(device).type(dtype=torch.float32)  #[1,12,1024,256]


In [ ]:
numbers = [1]
numbers.extend(range(100, 3001, 100))
for i in numbers:
    model = MyModel().to(device)
    optimizer = torch.optim.AdamW([{'params': model.parameters()}], 
                                   lr=0.00001, betas=(0.9, 0.999), weight_decay=0.0001)
    _, model, _, _ = load_state(model, optimizer, "./checkpoints-VAE-marmousi-G-N(0,0.1)-recheck2/checkpoint-VAE-marmousi-1-{}.pth".format(i)) 
    model = model.to(device)
    model_vgg = torchvision.models.vgg16(pretrained=True).features[:13].to(device) #第13层输出--conv5的结果。

    # 输出并保存每个epoch隐码的均值和标准差
    v_pred, Z_mean, log_Z_std = train(model, data, device)
    Z_std = torch.exp(0.5 * log_Z_std)
    
    torch.save(Z_mean, './VAE_train/Z_mean_VAE_GN(0,0.1)/Z_mean_epoch_{}'.format(i))
    torch.save(Z_std, './VAE_train/Z_std_VAE_GN(0,0.1)/Z_std_epoch_{}'.format(i))


In [ ]:
# n=1
numbers = [1]
numbers.extend(range(100, 3001, 1000))
for n in numbers:

    for i in range(0, 255):
        Z_mean = torch.load('./VAE_train/Z_mean_VAE_GN(0,1)/Z_mean_epoch_{}'.format(n))
        Z_std = torch.load('./VAE_train/Z_std_VAE_GN(0,1)/Z_std_epoch_{}'.format(n))
        # 定义均值和方差
        Z_mean_VAE_one = Z_mean[0,i]
        Z_std_VAE_one = Z_std[0,i]
        # 生成 x 值  
        z_VAE = np.linspace(-50, 50, 3000)

        # 计算正态分布的概率密度函数值
        pdf_VAE_Z = norm.pdf(z_VAE, Z_mean_VAE_one.cpu().detach().numpy(), Z_std_VAE_one.cpu().detach().numpy())
        plt.plot(z_VAE, pdf_VAE_Z, label='Z_VAE Distribution epoch {}'.format(i), linewidth=2, color='pink')

    z = np.linspace(-50, 50, 3000) 
    pdf_Z = norm.pdf(z, 0, 1)
    plt.plot(z, pdf_Z, label='normal Distribution epoch {}'.format(i), linewidth=2, color='red')

    # 在循环结束后添加图例，标签，标题等信息，并显示图像
    # plt.legend()
    plt.title('Normal Distribution Epoch {}'.format(n))
    plt.xlabel('V [m/s]')
    plt.ylabel('Density')
    plt.show()


In [ ]:
# n=1
numbers = [1]
numbers.extend(range(100, 3001, 100))
for n in numbers:

    for i in range(0, 255):
        Z_mean = torch.load('./VAE_train/Z_mean_VAE_GN(0,0.1)/Z_mean_epoch_{}'.format(n))
        Z_std = torch.load('./VAE_train/Z_std_VAE_GN(0,0.1)/Z_std_epoch_{}'.format(n))
        # 定义均值和方差
        Z_mean_VAE_one = Z_mean[0,i]
        Z_std_VAE_one = Z_std[0,i]
        # 生成 x 值  
        z_VAE = np.linspace(-50, 50, 3000)

        # 计算正态分布的概率密度函数值
        pdf_VAE_Z = norm.pdf(z_VAE, Z_mean_VAE_one.cpu().detach().numpy(), Z_std_VAE_one.cpu().detach().numpy())
        plt.plot(z_VAE, pdf_VAE_Z, label='Z_VAE Distribution epoch {}'.format(i), linewidth=2, color='pink')

    z = np.linspace(-50, 50, 3000) 
    pdf_Z = norm.pdf(z, 0, 0.01)
    plt.plot(z, pdf_Z, label='normal Distribution epoch {}'.format(i), linewidth=2, color='red')

    # 在循环结束后添加图例，标签，标题等信息，并显示图像
    # plt.legend()
    plt.title('Normal Distribution Epoch {}'.format(n))
    plt.xlabel('V [m/s]')
    plt.ylabel('Density')
    plt.show()


In [ ]:
# n=1
numbers = [1]
numbers.extend(range(100, 3001, 100))
for n in numbers:

    for i in range(0, 255):
        Z_mean = torch.load('./VAE_train/Z_mean_VAE_GN(0,0.01)/Z_mean_epoch_{}'.format(n))
        Z_std = torch.load('./VAE_train/Z_std_VAE_GN(0,0.01)/Z_std_epoch_{}'.format(n))
        # 定义均值和方差
        Z_mean_VAE_one = Z_mean[0,i]
        Z_std_VAE_one = Z_std[0,i]
        # 生成 x 值  
        z_VAE = np.linspace(-50, 50, 3000)

        # 计算正态分布的概率密度函数值
        pdf_VAE_Z = norm.pdf(z_VAE, Z_mean_VAE_one.cpu().detach().numpy(), Z_std_VAE_one.cpu().detach().numpy())
        plt.plot(z_VAE, pdf_VAE_Z, label='Z_VAE Distribution epoch {}'.format(i), linewidth=2, color='pink')

    z = np.linspace(-50, 50, 3000) 
    pdf_Z = norm.pdf(z, 0, 0.01)
    plt.plot(z, pdf_Z, label='normal Distribution epoch {}'.format(i), linewidth=2, color='red')

    # 在循环结束后添加图例，标签，标题等信息，并显示图像
    # plt.legend()
    plt.title('Normal Distribution Epoch {}'.format(n))
    plt.xlabel('V [m/s]')
    plt.ylabel('Density')
    plt.show()
    

In [ ]:
# n=100 #分布的位置 [0,255]

# numbers = [1]
# numbers.extend(range(100, 3001, 100))
# for i in numbers:
#     Z_mean = torch.load('./VAE_train/Z_mean_VAE_GN(0,0.1)/Z_mean_epoch_{}'.format(i))
#     Z_std = torch.load('./VAE_train/Z_std_VAE_GN(0,0.1)/Z_std_epoch_{}'.format(i))
#     # 定义均值和方差
#     Z_mean_VAE_one = Z_mean[0,n]
#     Z_std_VAE_one = Z_std[0,n]

#     # 生成 x 值  
#     z_VAE = np.linspace(-2, 2, 3000)
#     z = np.linspace(-2, 2, 3000)

#     # 计算正态分布的概率密度函数值
#     pdf_VAE_Z = norm.pdf(z_VAE, Z_mean_VAE_one.cpu().detach().numpy(), Z_std_VAE_one.cpu().detach().numpy())
#     pdf_Z = norm.pdf(z, 0, 0.01)

#     plt.plot(z_VAE, pdf_VAE_Z, label='Z_VAE Distribution', color='darkblue')
#     plt.plot(z, pdf_Z, label='normal Distribution', color='red')


#     plt.legend()
#     plt.title('Normal Distribution epoch {}'.format(i))
#     plt.xlabel('V [m/s]')
#     plt.ylabel('Density')
#     plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.manifold import MDS

# 加载示例数据集，这里以手写数字数据集为例
digits = load_digits()
data = digits.data
target = digits.target

# 初始化 MDS 对象。这里设置 n_components=2 以便我们可以在二维空间中进行可视化
mds = MDS(n_components=2, random_state=1)

# 对数据进行降维处理
data_transformed = mds.fit_transform(data)

# 对降维后的数据进行绘图
plt.scatter(data_transformed[:, 0], data_transformed[:, 1], c=target, cmap='Spectral', alpha=0.5)

# 添加色标以显示不同数字的颜色编码
plt.colorbar()

# 展示图表
plt.title('MDS of Digits Dataset')
plt.xlabel('MDS Dimension 1')
plt.ylabel('MDS Dimension 2')
plt.show()

In [ ]:
'''
多维缩放（MDS）是一种灵活的降维技术，可用于各种类型的数据集。MDS 旨在在低维空间中保留样本间的距离或相似度。对原始数据的维度，严格意义上讲，MDS 没有特定的要求。它可以应用于从几维到成千上万维的数据。
然而，原始数据的维度以及数据的性质确实对 MDS 分析的效果和可视化结果有重要影响。

高维数据集：高维数据是 MDS 常常处理的对象之一。在高维数据中，由于"维度的诅咒"，很难直观地理解数据结构。MDS 可以帮助通过将数据映射到二维或三维空间，来可视化这种复杂的多维数据集中的相似关系。
不同类型的数据：MDS 可以处理数字数据、分类数据等，因为 MDS 的关键在于数据点之间的相似度或距离矩阵。因此，只要能够合理定义样本间的距离或相似度，MDS 就可以应用。

数据的规模：尽管理论上没有限制，但是数据规模特别是样本大小，将影响 MDS 的计算和可视化效率。当处理非常大的数据集时，MDS 可能变得计算密集和时间消耗，特别是当计算全部样本之间距离的复杂度是平方级别的。

数据的预处理：对数据进行适当的预处理（如中心化、标准化）有时对于获得有意义的 MDS 结果是必要的，特别是当不同的特征有不同的度量单位和规模时。

距离计算：对于非数值数据，需要明确如何计算相似性或距离。例如，对于文本数据可能使用余弦相似度，而对于分类数据则可能需转换为适合的距离度量。

总之，MDS 不对原始数据的维度有硬性要求，但有效的数据处理、合理的距离（或相似度）度量选择以及对数据特性的考虑都是进行 MDS 分析时应该重视的因素。
'''

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

# 加载数据集，这里示例使用手写数字数据集
digits = load_digits()
data = digits.data
target = digits.target

# 初始化 t-SNE 对象，并指定目标维度为 2，以便在二维平面上进行可视化
tsne = TSNE(n_components=2, random_state=42)

# 对数据进行降维处理
data_tsne = tsne.fit_transform(data)

# 绘制降维后的数据
plt.scatter(data_tsne[:, 0], data_tsne[:, 1], c=target, cmap='Spectral', alpha=0.5)
plt.colorbar()  # 显示颜色条
plt.title('t-SNE visualization of Digits Dataset')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()

In [ ]:
'''
t-SNE（t-Distributed Stochastic Neighbor Embedding）是一种有效的高维数据降维技术，尤其适用于数据可视化。它能够捕捉数据的局部结构，并在低维空间中保留这些结构，使得相似的数据点在降维后的空间里彼此靠近，不相似的点则相距较远。
以下几点可以帮助你理解 t-SNE 的降维效果：
局部结构保留：t-SNE 设计之初就是为了保留数据的局部结构，也就是说在高维空间中距离近的数据点，在降维后的低维空间中应当尽量保持这种邻近关系。这样做通常可以揭示数据的团簇结构，即使在数据的全局结构非常复杂的情况下。

不等距聚类：t-SNE 对高维数据中不同聚类的相对位置和大小的还原不是非常准确，有时候它会过度拉开不同的团簇，使得我们不应把t-SNE的输出用于判断聚类之间的相对距离。

概率分布的转换：在 t-SNE 中，将高维空间中的数据转换成概率分布，然后试图在低维空间中重现相似的分布。在高维空间中，相邻点之间的概率由高斯分布决定；而在生成的低维空间中，它使用 t 分布来计算点之间的概率。
t 分布长尾的特性有助于优化过程在低维映射中正确区分局部相似的和不相似的数据点。

"拥挤问题"（Crowding Problem）的解决：t-SNE 试图解决传统方法（如PCA）中的拥挤问题，这是指在降低数据维度时，很难在低维空间中保留高维空间中的所有邻近关系。t 分布有助于确保低维空间中点与点之间有足够的间隔，这使得局部结构在可视化中更加明显。

超参数的选择：t-SNE 的主要超参数包括感知（perplexity）和学习率。这些参数的设置对结果有较大影响。感知参数可以被视为每个点在高维空间中考虑周围多少个近邻。学习率则影响到优化过程的速度和稳定性。

随机性：由于 t-SNE 启动过程中依赖于随机初始化，因此每次运行可能都会得到不同的结果，即使是同一个数据集。为了获取可重复的结果，需要设置随机数生成器的种子。

总而言之，理解 t-SNE 的降维效果需要重点关注它在可视化高维数据时如何在低维空间保留数据的局部结构。然而，t-SNE 更多提供了一种探索性的视窗，而非精确地再现数据的全局结构。因此，在使用 t-SNE 结果时，需要考虑其降维和数据可视化方面的限制。
'''